In [1744]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
import re
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from math import sqrt

# Census Data

In [1745]:
x2011 = pd.read_csv('zip_codes_2011.csv').drop(['do_date'], axis = 1).dropna(how = 'all', axis = 1)
x2012 = pd.read_csv('zip_codes_2012.csv').drop(['do_date'], axis = 1).dropna(how = 'all', axis = 1)
x2013 = pd.read_csv('zip_codes_2013.csv').drop(['do_date'], axis = 1).dropna(how = 'all', axis = 1)
x2014 = pd.read_csv('zip_codes_2014.csv').drop(['do_date'], axis = 1).dropna(how = 'all', axis = 1)
x2015 = pd.read_csv('zip_codes_2015.csv').drop(['do_date'], axis = 1).dropna(how = 'all', axis = 1)
x2016 = pd.read_csv('zip_codes_2016.csv').drop(['do_date'], axis = 1).dropna(how = 'all', axis = 1)
x2017 = pd.read_csv('zip_codes_2017.csv').drop(['do_date'], axis = 1).dropna(how = 'all', axis = 1)

x2011.columns = x2011.columns.str.replace('geo_id', 'zip')
x2012.columns = x2012.columns.str.replace('geo_id', 'zip')
x2013.columns = x2013.columns.str.replace('geo_id', 'zip')
x2014.columns = x2014.columns.str.replace('geo_id', 'zip')
x2015.columns = x2015.columns.str.replace('geo_id', 'zip')
x2016.columns = x2016.columns.str.replace('geo_id', 'zip')
x2017.columns = x2017.columns.str.replace('geo_id', 'zip')

mutual_columns = set(x2011.columns).intersection(set(x2012.columns)).intersection(set(x2013.columns)).\
intersection(set(x2014.columns)).intersection(set(x2015.columns)).intersection(set(x2016.columns)).\
intersection(set(x2017.columns))


x2011 = x2011[mutual_columns].sort_values('zip').set_index('zip')
x2012 = x2012[mutual_columns].sort_values('zip').set_index('zip')
x2013 = x2013[mutual_columns].sort_values('zip').set_index('zip')
x2014 = x2014[mutual_columns].sort_values('zip').set_index('zip')
x2015 = x2015[mutual_columns].sort_values('zip').set_index('zip')
x2016 = x2016[mutual_columns].sort_values('zip').set_index('zip')
x2017 = x2017[mutual_columns].sort_values('zip').set_index('zip')

In [1746]:
x2011.columns = [i + '_2011' for i in x2011.columns]
x2012.columns = [i + '_2012' for i in x2012.columns] 
x2013.columns = [i + '_2013' for i in x2013.columns]
x2014.columns = [i + '_2014' for i in x2014.columns]
x2015.columns = [i + '_2015' for i in x2015.columns]
x2016.columns = [i + '_2016' for i in x2016.columns]
x2017.columns = [i + '_2017' for i in x2017.columns]

In [1747]:
census2013 = pd.concat([x2013,x2012,x2011], join = 'inner', axis = 1).reset_index()
census2014 = pd.concat([x2014,x2013,x2012], join = 'inner', axis = 1).reset_index()
census2015 = pd.concat([x2015,x2014,x2013], join = 'inner', axis = 1).reset_index()
census2016 = pd.concat([x2016,x2015,x2014], join = 'inner', axis = 1).reset_index()
census2017 = pd.concat([x2017,x2016,x2015], join = 'inner', axis = 1).reset_index()

In [1748]:
census2013['year'] = '2013'
census2014['year'] = '2014'
census2015['year'] = '2015'
census2016['year'] = '2016'
census2017['year'] = '2017'

In [1749]:
census2013.columns = census2013.columns.str.replace('_2013','_3y').str.replace('_2012','_4y').str.replace('_2011','_5y')
census2014.columns = census2014.columns.str.replace('_2014','_3y').str.replace('_2013','_4y').str.replace('_2012','_5y')
census2015.columns = census2015.columns.str.replace('_2015','_3y').str.replace('_2014','_4y').str.replace('_2013','_5y')
census2016.columns = census2016.columns.str.replace('_2016','_3y').str.replace('_2015','_4y').str.replace('_2014','_5y')
census2017.columns = census2017.columns.str.replace('_2017','_3y').str.replace('_2016','_4y').str.replace('_2015','_5y')

In [1750]:
census = pd.concat([census2013,census2014, census2015, census2016, census2017]).sort_values(['zip','year'])


In [2201]:
(census.isna().sum(axis = 0) > 0)

zip                                False
pop_in_labor_force_3y              False
population_3_years_over_3y         False
female_65_to_66_3y                 False
rent_10_to_15_percent_3y           False
                                   ...  
female_22_to_24_5y                 False
rent_over_50_percent_5y            False
poverty_5y                         False
households_retirement_income_5y    False
year                               False
Length: 701, dtype: bool

# Zillow Rent Index

In [1752]:
zillow = pd.read_csv('Zip_Zri_AllHomesPlusMultifamily.csv')
zillow.columns = zillow.columns.str.replace('RegionName', 'zip')

zri2011 = pd.concat([zillow.zip,zillow.iloc[:,11:23]], axis = 1).set_index('zip').dropna(how = 'all')
zri2012 = pd.concat([zillow.zip,zillow.iloc[:,23:35]], axis = 1).set_index('zip').dropna(how = 'all')
zri2013 = pd.concat([zillow.zip,zillow.iloc[:,35:47]], axis = 1).set_index('zip').dropna(how = 'all')
zri2014 = pd.concat([zillow.zip,zillow.iloc[:,47:59]], axis = 1).set_index('zip').dropna(how = 'all')
zri2015 = pd.concat([zillow.zip,zillow.iloc[:,59:71]], axis = 1).set_index('zip').dropna(how = 'all')
zri2016 = pd.concat([zillow.zip,zillow.iloc[:,71:83]], axis = 1).set_index('zip').dropna(how = 'all')
zri2017 = pd.concat([zillow.zip,zillow.iloc[:,83:95]], axis = 1).set_index('zip').dropna(how = 'all')
zri2018 = pd.concat([zillow.zip,zillow.iloc[:,95:107]], axis = 1).set_index('zip').dropna(how = 'all')
zri2019 = pd.concat([zillow.zip,zillow.iloc[:,107:119]], axis = 1).set_index('zip').dropna(how = 'all')

zri2011 = zri2011[zri2011.isna().sum(axis = 1) <= 3]
zri2012 = zri2012[zri2012.isna().sum(axis = 1) <= 3]
zri2013 = zri2013[zri2013.isna().sum(axis = 1) <= 3]
zri2014 = zri2014[zri2014.isna().sum(axis = 1) <= 3]
zri2015 = zri2015[zri2015.isna().sum(axis = 1) <= 3]
zri2016 = zri2016[zri2016.isna().sum(axis = 1) <= 3]
zri2017 = zri2017[zri2017.isna().sum(axis = 1) <= 3]
zri2018 = zri2018[zri2018.isna().sum(axis = 1) <= 3]
zri2019 = zri2019[zri2019.isna().sum(axis = 1) <= 3]

In [1753]:
zri2011 = zri2011.interpolate(method='linear',limit_direction='both', axis=1)
zri2012 = zri2012.interpolate(method='linear',limit_direction='both', axis=1)
zri2013 = zri2013.interpolate(method='linear',limit_direction='both', axis=1)
zri2014 = zri2014.interpolate(method='linear',limit_direction='both', axis=1)
zri2015 = zri2015.interpolate(method='linear',limit_direction='both', axis=1)
zri2016 = zri2016.interpolate(method='linear',limit_direction='both', axis=1)
zri2017 = zri2017.interpolate(method='linear',limit_direction='both', axis=1)
zri2018 = zri2018.interpolate(method='linear',limit_direction='both', axis=1)
zri2019 = zri2019.interpolate(method='linear',limit_direction='both', axis=1)

In [2116]:
zillow_monthly = pd.concat([zri2011, zri2012, zri2013, zri2014, zri2015, zri2016, zri2017, zri2018, zri2019], join = 'inner', axis = 1)



zillow_monthly.shape[0]




8752

In [2118]:
zillow.shape[0]

13181

In [1755]:
zri2011 = pd.DataFrame(zri2011.mean(axis = 1), columns = ['2011'])
zri2012 = pd.DataFrame(zri2012.mean(axis = 1), columns = ['2012'])
zri2013 = pd.DataFrame(zri2013.mean(axis = 1), columns = ['2013'])
zri2014 = pd.DataFrame(zri2014.mean(axis = 1), columns = ['2014'])
zri2015 = pd.DataFrame(zri2015.mean(axis = 1), columns = ['2015'])
zri2016 = pd.DataFrame(zri2016.mean(axis = 1), columns = ['2016'])
zri2017 = pd.DataFrame(zri2017.mean(axis = 1), columns = ['2017'])
zri2018 = pd.DataFrame(zri2018.mean(axis = 1), columns = ['2018'])
zri2019 = pd.DataFrame(zri2019.mean(axis = 1), columns = ['2019'])

### Response (Target Variable)

In [2111]:
zillow_mean = pd.concat([zri2013,zri2014,zri2015,zri2016,zri2017,zri2018,zri2019],join = 'inner', axis = 1).reset_index()
zillow_mean = pd.melt(zillow_mean, id_vars=['zip'], value_vars=['2013','2014','2015','2016','2017','2018','2019'],\
       var_name='year', value_name='ZRI+3').sort_values(['zip','year'])



In [2115]:
zillow_mean.shape[0]/7

9863.0

In [1757]:
zillow_mean = zillow_mean[(zillow_mean['year'] == '2016') | (zillow_mean['year'] == '2017') | (zillow_mean['year'] == '2018') | (zillow_mean['year'] == '2019')]


In [1758]:
zillow_mean['year'] = zillow_mean.year.apply(lambda i:str(int(i)-3))

In [1759]:
zillow_mean

,zip,year,ZRI+3
34898,1001,2013,1380.166667
44761,1001,2014,1421.833333
54624,1001,2015,1338.500000
64487,1001,2016,1362.416667
38790,1005,2013,1268.833333
...,...,...,...
64639,99701,2016,1506.416667
34516,99705,2013,1502.666667
44379,99705,2014,1474.750000
54242,99705,2015,1479.583333


### Train Data (Zillow Rent Index from previous Years)

In [1760]:
zri_train_2013 = pd.concat([zri2013,zri2014,zri2015], join = 'inner', axis = 1)
zri_train_2014 = pd.concat([zri2014,zri2015,zri2016], join = 'inner', axis = 1)
zri_train_2015 = pd.concat([zri2015,zri2016,zri2017], join = 'inner', axis = 1)
zri_train_2016 = pd.concat([zri2016,zri2017,zri2018], join = 'inner', axis = 1)

In [1761]:
zri_train_2013.columns = zri_train_2013.columns.str.replace('2015','zri_1y').str.replace('2014','zri_2y').str.replace('2013','zri_3y')
zri_train_2014.columns = zri_train_2014.columns.str.replace('2016','zri_1y').str.replace('2015','zri_2y').str.replace('2014','zri_3y')
zri_train_2015.columns = zri_train_2015.columns.str.replace('2017','zri_1y').str.replace('2016','zri_2y').str.replace('2015','zri_3y')
zri_train_2016.columns = zri_train_2016.columns.str.replace('2018','zri_1y').str.replace('2017','zri_2y').str.replace('2016','zri_3y')

In [1762]:
zri_train_2013['year'] = '2013'
zri_train_2014['year'] = '2014'
zri_train_2015['year'] = '2015'
zri_train_2016['year'] = '2016'

In [1763]:
zri_train_2013 = zri_train_2013.reset_index()
zri_train_2014 = zri_train_2014.reset_index()
zri_train_2015 = zri_train_2015.reset_index()
zri_train_2016 = zri_train_2016.reset_index()

In [1764]:
zillow_train = pd.concat([zri_train_2013,zri_train_2014, zri_train_2015, zri_train_2016])


zillow_train

,zip,zri_3y,zri_2y,zri_1y,year
0,60657,1955.666667,2005.666667,2073.500000,2013
1,77494,1786.833333,1956.833333,1921.916667,2013
2,60614,2148.000000,2202.583333,2271.000000,2013
3,77449,1251.166667,1361.083333,1424.000000,2013
4,77084,1205.916667,1327.916667,1417.166667,2013
...,...,...,...,...,...
12043,26291,1021.583333,949.666667,918.416667,2016
12044,12720,1196.500000,1181.416667,1261.250000,2016
12045,26209,732.666667,762.666667,765.083333,2016
12046,41101,865.083333,851.666667,855.916667,2016


In [2212]:
total = zillow_train.merge(census, how = 'inner', on = ['zip','year']).merge(zillow_mean, how = 'inner', on = ['zip','year'])



In [2209]:
total = total.dropna(axis = 0).sort_values(['zip','year'])

In [2225]:
total.shape

(39404, 705)

In [2229]:
total.isna().sum(axis = 0)

zip                                0
zri_3y                             0
zri_2y                             0
zri_1y                             0
year                               0
                                  ..
female_22_to_24_5y                 0
rent_over_50_percent_5y            0
poverty_5y                         0
households_retirement_income_5y    0
ZRI+3                              0
Length: 705, dtype: int64

In [2224]:
total.dropna(how = 'all',axis = 1).shape

(39404, 705)

# Machine Learning

## Baseline Model (just Zillow Rent Indexes from previous years)

In [1768]:
base_train = zri_train_2015.merge(zillow_mean, how = 'inner', on = ['zip','year'])
base_test = zri_train_2016.merge(zillow_mean, how = 'inner', on = ['zip','year'])

In [1769]:
base_x_train = base_train.iloc[:,1:4]
base_x_test = base_test.iloc[:,1:4]

In [1770]:
base_y_train = base_train.iloc[:,5]
base_y_test = base_test.iloc[:,5]

### Log Everything

In [1771]:
base_x_train = np.log(base_x_train+1)
base_x_test = np.log(base_x_test+1)
base_y_train = np.log(base_y_train)
base_y_test = np.log(base_y_test)

In [1772]:
base_model_ols = LinearRegression()
base_model_lasso = Lasso()
base_model_ridge = Ridge()
base_model_gradient = GradientBoostingRegressor()

In [1773]:
base_model_ols.fit(base_x_train, base_y_train)
base_model_lasso.fit(base_x_train, base_y_train)
base_model_ridge.fit(base_x_train, base_y_train)
base_model_gradient.fit(base_x_train, base_y_train)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
                          learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='auto',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

### Train Score

In [1774]:
print('Simple Linear Train Score: %.4f'%model_ols.score(x_train, y_train))
print('Lasso Train Score: %.4f'%model_lasso.score(x_train, y_train))
print('Ridge Train Score: %.4f'%model_ridge.score(x_train, y_train))
print('Gradient Regressor Train Score: %.4f'%model_gradient.score(x_train, y_train))

Simple Linear Train Score: 0.9939
Lasso Train Score: 0.0000
Ridge Train Score: 0.9939
Gradient Regressor Train Score: 0.9939


### Test Score

In [1775]:
print('Simple Linear Test Score: %.4f'%base_model_ols.score(base_x_test, base_y_test))
print('Lasso Test Score: %.4f'%base_model_lasso.score(base_x_test, base_y_test))
print('Ridge Test Score: %.4f'%base_model_ridge.score(base_x_test, base_y_test))
print('Gradient Regressor Test Score: %.4f'%base_model_gradient.score(base_x_test, base_y_test))

Simple Linear Test Score: 0.9921
Lasso Test Score: -0.0007
Ridge Test Score: 0.9919
Gradient Regressor Test Score: 0.9917


### RMSE Score

In [2051]:
print('Simple Linear RMSE Score: %.5f' % sqrt(mean_squared_error(np.exp(base_model_ols.predict(base_x_test)), np.exp(base_y_test))))
print('Lasso RMSE Score: %.5f' % sqrt(mean_squared_error(np.exp(base_model_lasso.predict(base_x_test)), np.exp(base_y_test))))
print('Ridge RMSE Score: %.5f' % sqrt(mean_squared_error(np.exp(base_model_ridge.predict(base_x_test)), np.exp(base_y_test))))
print('Gradient Regressor RMSE Score: %.5f' % sqrt(mean_squared_error(np.exp(base_model_gradient.predict(base_x_test)), np.exp(base_y_test))))


Simple Linear RMSE Score: 48.93983
Lasso RMSE Score: 610.14751
Ridge RMSE Score: 49.25927
Gradient Regressor RMSE Score: 51.05961


### Prediction Accuracy

In [1782]:
base_pred = pd.Series(np.exp(base_model_ols.predict(base_x_test)), index = base_y_test.index)
print("Average Prediction Accuracy: %.2f percent" %((1 - np.abs(np.exp(base_y_test)/base_pred-1)).mean() * 100))

Average Prediction Accuracy: 97.61 percent


## Census Model

In [1790]:
train = total[total['year'] != '2016']
train['zip'] = [str(i) for i in train.zip]
train['zip_year'] = train.zip + '_' + train.year
train = train.drop(['zip','year'], axis = 1).set_index('zip_year')

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [1791]:
test = total[total['year'] == '2016']
test['zip'] = [str(i) for i in test.zip]
test['zip_year'] = test.zip + '_' + test.year
test = test.drop(['zip','year'], axis = 1).set_index('zip_year')

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [1792]:
x_train = train.iloc[:,:-1]
y_train = train.iloc[:,-1]

In [1793]:
x_test = test.iloc[:,:-1]
y_test = test.iloc[:,-1]

### Log Everything

In [1794]:
x_train = np.log(x_train+1)
x_test = np.log(x_test+1)
y_train = np.log(y_train)
y_test = np.log(y_test)

In [1795]:
model_ols = LinearRegression()
model_lasso = Lasso()
model_ridge = Ridge()
model_gradient = GradientBoostingRegressor()

In [1796]:
model_ols.fit(x_train, y_train)
model_lasso.fit(x_train, y_train)
model_ridge.fit(x_train, y_train)
model_gradient.fit(x_train, y_train)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
                          learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='auto',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

### Train Score

In [1799]:
print('Simple Linear Train Score: %.4f'%model_ols.score(x_train, y_train))
print('Lasso Train Score: %.4f'%model_lasso.score(x_train, y_train))
print('Ridge Train Score: %.4f'%model_ridge.score(x_train, y_train))
print('Gradient Regressor Train Score: %.4f'%model_gradient.score(x_train, y_train))

Simple Linear Train Score: 0.9939
Lasso Train Score: 0.0000
Ridge Train Score: 0.9939
Gradient Regressor Train Score: 0.9939


### Test Score

In [1807]:
print('Simple Linear Test Score: %.4f'%model_ols.score(x_test, y_test))
print('Lasso Test Score: %.4f'%model_lasso.score(x_test, y_test))
print('Ridge Test Score: %.4f'%model_ridge.score(x_test, y_test))
print('Gradient Regressor Test Score: %.4f'%model_gradient.score(x_test, y_test))

Simple Linear Test Score: 0.9922
Lasso Test Score: -0.0030
Ridge Test Score: 0.9920
Gradient Regressor Test Score: 0.9916


### RMSE

In [1810]:
print('Simple Linear RMSE Score: %.5f' % sqrt(mean_squared_error(np.exp(model_ols.predict(x_test)), np.exp(y_test))))
print('Lasso RMSE Score: %.5f' % sqrt(mean_squared_error(np.exp(model_lasso.predict(x_test)), np.exp(y_test))))
print('Ridge RMSE Score: %.5f' % sqrt(mean_squared_error(np.exp(model_ridge.predict(x_test)), np.exp(y_test))))
print('Gradient Regressor RMSE Score: %.5f' % sqrt(mean_squared_error(np.exp(model_gradient.predict(x_test)), np.exp(y_test))))



Simple Linear Kaggle Score: 50.41683
Lasso Kaggle Score: 613.52357
Ridge Kaggle Score: 51.26327
Gradient Regressor Kaggle Score: 50.41806


### Prediction Accuracy

In [1159]:
pred = pd.Series(np.exp(model_ols.predict(x_test)), index = y_test.index)

In [1160]:
print("Average Prediction Accuracy: %.2f percent" %((1 - np.abs(np.exp(y_test)/pred-1)).mean() * 100))

Average Prediction Accuracy: 97.61 percent


## Feature Importance

In [2062]:
top_features = sorted(list(zip(x_train, model_gradient.feature_importances_)), key = lambda x: x[1], reverse = True)[0:10]

top_features

[('zri_1y', 0.9954888527172195),
 ('owner_occupied_housing_units_median_value_5y', 0.001247875699917956),
 ('owner_occupied_housing_units_upper_value_quartile_5y',
  0.0011228900236242769),
 ('median_rent_5y', 0.00035388129505331306),
 ('median_rent_4y', 0.0002619130855406447),
 ('housing_built_2005_or_later_5y', 0.00023572833615593402),
 ('median_rent_3y', 0.00017116410379575684),
 ('housing_built_2005_or_later_4y', 0.00015083549834439276),
 ('income_per_capita_3y', 0.00011276525743124804),
 ('percent_income_spent_on_rent_3y', 7.78262098348698e-05)]

## Model With Just Top 10 Features Census

In [2063]:
top_x_train = x_train[[i[0] for i in top_features] + ['zri_2y'] + ['zri_3y']]
top_x_test = x_test[[i[0] for i in top_features] + ['zri_2y'] + ['zri_3y']]

In [2064]:
top_y_train = y_train.copy()
top_y_test = y_test.copy()

In [2065]:
top_model_ols = LinearRegression()
top_model_lasso = Lasso()
top_model_ridge = Ridge()
top_model_gradient = GradientBoostingRegressor()

top_model_ols.fit(top_x_train, top_y_train)
top_model_lasso.fit(top_x_train, top_y_train)
top_model_ridge.fit(top_x_train, top_y_train)
top_model_gradient.fit(top_x_train, top_y_train)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
                          learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='auto',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

### Train Score

In [2066]:
print('Simple Linear Train Score: %.4f'%top_model_ols.score(top_x_train, top_y_train))
print('Lasso Train Score: %.4f'%top_model_lasso.score(top_x_train, top_y_train))
print('Ridge Train Score: %.4f'%top_model_ridge.score(top_x_train, top_y_train))
print('Gradient Regressor Train Score: %.4f'%top_model_gradient.score(top_x_train, top_y_train))

Simple Linear Train Score: 0.9934
Lasso Train Score: 0.0000
Ridge Train Score: 0.9934
Gradient Regressor Train Score: 0.9936


### Test Score

In [2067]:
print('Simple Linear Test Score: %.4f'%top_model_ols.score(top_x_test, top_y_test))
print('Lasso Test Score: %.4f'%top_model_lasso.score(top_x_test, top_y_test))
print('Ridge Test Score: %.4f'%top_model_ridge.score(top_x_test, top_y_test))
print('Gradient Regressor Test Score: %.4f'%top_model_gradient.score(top_x_test, top_y_test))

Simple Linear Test Score: 0.9923
Lasso Test Score: -0.0030
Ridge Test Score: 0.9922
Gradient Regressor Test Score: 0.9915


### RMSE Score

In [2100]:
print('Simple Linear RMSE Score: %.5f' % sqrt(mean_squared_error(np.exp(top_model_ols.predict(top_x_test)), np.exp(top_y_test))))
print('Lasso RMSE Score: %.5f' % sqrt(mean_squared_error(np.exp(top_model_lasso.predict(top_x_test)), np.exp(top_y_test))))
print('Ridge RMSE Score: %.5f' % sqrt(mean_squared_error(np.exp(top_model_ridge.predict(top_x_test)), np.exp(top_y_test))))
print('Gradient Regressor RMSE Score: %.5f' % sqrt(mean_squared_error(np.exp(top_model_gradient.predict(top_x_test)), np.exp(top_y_test))))



Simple Linear RMSE Score: 47.63700
Lasso RMSE Score: 613.52357
Ridge RMSE Score: 47.84701
Gradient Regressor RMSE Score: 50.51091


,zri_3y,zri_2y,zri_1y,pop_in_labor_force_3y,population_3_years_over_3y,female_65_to_66_3y,rent_10_to_15_percent_3y,family_households_3y,female_60_to_61_3y,rent_20_to_25_percent_3y,...,female_67_to_69_5y,male_18_to_19_5y,occupation_sales_office_5y,two_or_more_races_pop_5y,owner_occupied_housing_units_lower_value_quartile_5y,income_per_capita_5y,female_22_to_24_5y,rent_over_50_percent_5y,poverty_5y,households_retirement_income_5y
zip_year,,,,,,,,,,,,,,,,,,,,,
1001_2016,7.230684,7.260405,7.200052,9.128262,9.742732,5.564520,5.135798,8.279951,4.997212,5.298317,...,5.379897,5.075174,7.769801,4.897840,12.028745,10.290279,6.030685,6.398595,7.237778,7.123673
1005_2016,7.146641,7.157606,7.157606,8.002694,8.548110,4.553877,2.833213,7.048386,4.343805,4.060443,...,3.828641,4.653960,6.439350,0.693147,11.993504,10.275224,3.784190,4.189655,5.416100,5.638355
1007_2016,7.331879,7.307286,7.316880,9.058820,9.583213,5.303305,4.369448,8.296796,5.525453,3.737670,...,4.890349,5.117994,7.365180,5.746203,12.189948,10.410727,5.247024,5.831882,7.037028,6.905753
1013_2016,7.056247,7.112667,7.098307,9.348536,9.999298,5.267858,5.743003,8.618305,5.808142,6.226537,...,5.398163,6.142037,7.889459,5.583496,11.884496,9.969837,6.495266,6.862758,8.123854,7.290293
1020_2016,7.072281,7.126690,7.095272,9.726989,10.303940,6.091310,5.645447,8.994545,6.109248,6.242223,...,6.021023,5.863631,8.295049,6.311735,11.940811,10.224484,6.063785,6.538140,8.063378,7.872836
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99577_2016,7.430954,7.396386,7.405597,9.626086,10.195635,5.446737,4.430817,8.858937,5.843544,5.631212,...,5.036953,5.953243,7.898411,7.323831,12.427218,10.556073,6.376727,6.104793,6.801283,7.716906
99645_2016,7.295170,7.270893,7.248623,9.519735,10.203481,5.480639,5.402677,8.728588,5.905362,5.393628,...,5.717028,6.152733,7.898411,7.498870,12.075400,10.354563,5.998937,5.958425,7.642044,7.503290
99654_2016,7.284992,7.257061,7.251877,10.249167,10.939817,6.025866,5.849325,9.473012,6.539586,6.437752,...,5.955837,6.658011,8.602820,8.179200,12.032915,10.304141,6.830874,7.021084,8.606485,8.142645


In [2139]:
 k = list(map(lambda x: str(x), top_y_test.reset_index()['zip_year']))

In [2152]:
k_2 = list(map(lambda x: x.split('_')[0], k))

In [2154]:
for i in range(len(k_2)):
    if len(k_2[i]) == 4:
        k_2[i] = '0' + k_2[i]

In [2174]:
k_3 = list(map(lambda x: int(x), k_2))

In [2186]:
n = ['01001', '01005', '01007', '01013', '01020', '01028', '01030', '01040', '01056', '01057', '01060', '01075', '01077', '01085', '01089', '01104', '01105', '01107', '01108', '01109', '01118', '01119', '01129', '01151', '01331', '01420', '01430', '01432', '01440', '01450', '01452', '01453', '01460', '01462', '01463', '01464', '01469', '01473', '01475', '01501', '01504', '01505', '01506', '01507', '01510', '01516', '01518', '01519', '01520', '01522', '01523', '01524', '01527', '01529', '01532', '01534', '01535', '01540', '01541', '01550', '01560', '01562', '01564', '01569', '01571', '01585', '01588', '01590', '01602', '01603', '01604', '01605', '01606', '01607', '01609', '01610', '01702', '01720', '01730', '01740', '01741', '01746', '01747', '01748', '01754', '01756', '01757', '01770', '01773', '01775', '01776', '01778', '01801', '01803', '01810', '01821', '01824', '01826', '01830', '01832', '01833', '01834', '01835', '01841', '01843', '01844', '01845', '01850', '01851', '01852', '01854', '01860', '01862', '01863', '01864', '01867', '01876', '01879', '01880', '01886', '01887', '01890', '01902', '01905', '01906', '01907', '01908', '01913', '01915', '01930', '01938', '01940', '01945', '01949', '01950', '01952', '01960', '01966', '01969', '01970', '01982', '01984', '02019', '02021', '02026', '02030', '02032', '02035', '02038', '02043', '02045', '02050', '02052', '02053', '02056', '02062', '02066', '02067', '02072', '02081', '02093', '02108', '02109', '02111', '02113', '02114', '02115', '02116', '02118', '02119', '02120', '02121', '02122', '02124', '02125', '02127', '02128', '02129', '02130', '02134', '02135', '02136', '02138', '02139', '02140', '02141', '02143', '02144', '02145', '02148', '02149', '02155', '02169', '02170', '02171', '02176', '02180', '02184', '02186', '02188', '02189', '02190', '02191', '02210', '02215', '02301', '02302', '02322', '02324', '02332', '02333', '02338', '02339', '02343', '02346', '02347', '02351', '02359', '02360', '02364', '02370', '02379', '02382', '02420', '02421', '02445', '02446', '02451', '02459', '02460', '02467', '02468', '02472', '02474', '02476', '02478', '02482', '02492', '02493', '02532', '02536', '02537', '02538', '02563', '02571', '02576', '02601', '02632', '02638', '02644', '02648', '02649', '02660', '02664', '02673', '02675', '02703', '02718', '02719', '02726', '02738', '02739', '02740', '02743', '02744', '02747', '02764', '02767', '02769', '02770', '02771', '02777', '02780', '02790', '02813', '02814', '02816', '02817', '02818', '02822', '02828', '02830', '02831', '02832', '02835', '02837', '02840', '02842', '02852', '02857', '02860', '02861', '02863', '02864', '02865', '02871', '02878', '02886', '02888', '02889', '02891', '02893', '02895', '02896', '02903', '02904', '02905', '02906', '02907', '02908', '02909', '02910', '02911', '02914', '02915', '02916', '02917', '02919', '02920', '03031', '03032', '03034', '03037', '03038', '03042', '03044', '03045', '03052', '03053', '03054', '03060', '03063', '03064', '03070', '03071', '03077', '03079', '03087', '03102', '03104', '03106', '03109', '03110', '03234', '03237', '03244', '03246', '03249', '03275', '03276', '03281', '03301', '03303', '03304', '03431', '03446', '03470', '03584', '03598', '03602', '03766', '03782', '03801', '03811', '03819', '03833', '03835', '03836', '03839', '03841', '03842', '03848', '03851', '03857', '03858', '03861', '03862', '03865', '03867', '03873', '03874', '03878', '03885', '04011', '04038', '04092', '04097', '04102', '04103', '04106', '04260', '04268', '04330', '04364', '04401', '04412', '04416', '04426', '04538', '04578', '04915', '04938', '04963', '06001', '06002', '06010', '06019', '06026', '06029', '06032', '06033', '06035', '06037', '06040', '06042', '06051', '06053', '06057', '06062', '06063', '06066', '06067', '06070', '06073', '06074', '06076', '06078', '06082', '06084', '06085', '06088', '06089', '06093', '06095', '06096', '06098', '06105', '06106', '06107', '06108', '06109', '06110', '06111', '06112', '06114', '06117', '06118', '06119', '06120', '06231', '06234', '06238', '06239', '06241', '06248', '06249', '06250', '06259', '06260', '06279', '06320', '06330', '06331', '06333', '06340', '06351', '06354', '06355', '06357', '06359', '06360', '06365', '06370', '06371', '06374', '06382', '06384', '06385', '06401', '06403', '06405', '06410', '06413', '06415', '06416', '06417', '06418', '06420', '06423', '06424', '06437', '06442', '06443', '06447', '06450', '06451', '06457', '06460', '06461', '06468', '06469', '06470', '06471', '06472', '06473', '06475', '06477', '06478', '06479', '06482', '06483', '06484', '06488', '06489', '06492', '06498', '06511', '06512', '06513', '06514', '06515', '06516', '06517', '06518', '06519', '06525', '06604', '06605', '06606', '06607', '06608', '06610', '06611', '06615', '06704', '06705', '06706', '06708', '06710', '06712', '06716', '06751', '06757', '06762', '06763', '06770', '06776', '06782', '06784', '06786', '06787', '06790', '06791', '06795', '06798', '06801', '06804', '06807', '06810', '06811', '06812', '06820', '06824', '06825', '06830', '06840', '06850', '06851', '06854', '06855', '06870', '06877', '06878', '06880', '06883', '06890', '06897', '06901', '06902', '06903', '06905', '06906', '06907', '07001', '07002', '07003', '07004', '07005', '07006', '07008', '07010', '07011', '07012', '07013', '07014', '07016', '07017', '07018', '07020', '07022', '07024', '07026', '07028', '07030', '07032', '07033', '07035', '07036', '07040', '07042', '07043', '07044', '07045', '07047', '07050', '07054', '07055', '07057', '07059', '07065', '07066', '07067', '07070', '07071', '07072', '07073', '07075', '07079', '07080', '07081', '07083', '07086', '07087', '07090', '07093', '07094', '07095', '07103', '07104', '07105', '07106', '07107', '07108', '07109', '07110', '07111', '07112', '07114', '07201', '07202', '07203', '07204', '07205', '07206', '07208', '07302', '07304', '07305', '07306', '07307', '07403', '07407', '07410', '07416', '07419', '07420', '07421', '07424', '07430', '07435', '07436', '07438', '07440', '07442', '07446', '07450', '07452', '07456', '07457', '07461', '07462', '07463', '07465', '07470', '07480', '07481', '07502', '07506', '07508', '07522', '07601', '07603', '07605', '07607', '07621', '07624', '07628', '07630', '07631', '07640', '07642', '07643', '07644', '07645', '07646', '07647', '07649', '07650', '07652', '07656', '07657', '07660', '07661', '07662', '07663', '07666', '07670', '07675', '07676', '07701', '07712', '07716', '07718', '07719', '07721', '07722', '07724', '07726', '07727', '07728', '07730', '07731', '07732', '07734', '07735', '07740', '07746', '07747', '07748', '07750', '07751', '07753', '07756', '07758', '07760', '07764', '07801', '07821', '07823', '07826', '07827', '07828', '07830', '07834', '07840', '07843', '07848', '07849', '07852', '07853', '07856', '07860', '07863', '07866', '07869', '07871', '07874', '07876', '07882', '07885', '07901', '07920', '07921', '07922', '07924', '07927', '07928', '07932', '07933', '07936', '07940', '07945', '07950', '07960', '07974', '07981', '08002', '08003', '08004', '08005', '08007', '08009', '08010', '08012', '08015', '08016', '08021', '08022', '08027', '08028', '08029', '08030', '08031', '08033', '08034', '08035', '08037', '08043', '08045', '08046', '08048', '08050', '08051', '08052', '08053', '08054', '08055', '08057', '08060', '08061', '08062', '08063', '08065', '08066', '08068', '08069', '08070', '08071', '08075', '08077', '08078', '08079', '08080', '08081', '08083', '08084', '08085', '08086', '08087', '08088', '08091', '08092', '08094', '08096', '08097', '08098', '08103', '08104', '08107', '08108', '08109', '08110', '08201', '08203', '08205', '08215', '08232', '08234', '08302', '08312', '08318', '08322', '08330', '08332', '08343', '08344', '08360', '08361', '08401', '08501', '08502', '08505', '08512', '08514', '08515', '08518', '08520', '08527', '08530', '08533', '08534', '08535', '08536', '08540', '08550', '08551', '08554', '08559', '08609', '08610', '08618', '08619', '08628', '08629', '08638', '08648', '08690', '08691', '08701', '08721', '08722', '08723', '08724', '08730', '08731', '08734', '08736', '08740', '08741', '08742', '08750', '08753', '08755', '08758', '08759', '08801', '08802', '08804', '08805', '08807', '08809', '08810', '08812', '08816', '08817', '08820', '08822', '08823', '08824', '08825', '08827', '08828', '08829', '08830', '08831', '08833', '08835', '08837', '08840', '08844', '08846', '08848', '08852', '08854', '08857', '08859', '08861', '08863', '08865', '08869', '08873', '08876', '08879', '08880', '08882', '08884', '08886', '08889', '08901', '08902', '08904', '10004', '10005', '10018', '10035', '10301', '10302', '10303', '10304', '10305', '10306', '10310', '10314', '10453', '10455', '10456', '10457', '10458', '10459', '10460', '10461', '10462', '10463', '10464', '10465', '10466', '10467', '10468', '10469', '10470', '10471', '10472', '10473', '10502', '10504', '10509', '10510', '10512', '10514', '10522', '10523', '10528', '10530', '10533', '10536', '10538', '10541', '10543', '10547', '10549', '10552', '10560', '10562', '10566', '10570', '10573', '10576', '10583', '10589', '10591', '10595', '10598', '10601', '10603', '10604', '10605', '10606', '10607', '10701', '10703', '10704', '10705', '10706', '10707', '10708', '10710', '10801', '10803', '10804', '10805', '10901', '10916', '10918', '10921', '10923', '10924', '10925', '10926', '10927', '10928', '10940', '10941', '10950', '10954', '10958', '10960', '10962', '10965', '10968', '10970', '10974', '10977', '10983', '10987', '10990', '10992', '10993', '10998', '11001', '11003', '11004', '11010', '11021', '11023', '11030', '11040', '11050', '11096', '11101', '11102', '11103', '11104', '11105', '11106', '11201', '11203', '11204', '11205', '11206', '11207', '11208', '11209', '11210', '11211', '11212', '11213', '11214', '11215', '11216', '11217', '11218', '11219', '11220', '11221', '11222', '11223', '11224', '11225', '11226', '11228', '11229', '11230', '11231', '11232', '11233', '11235', '11238', '11354', '11355', '11356', '11357', '11358', '11360', '11361', '11362', '11363', '11364', '11365', '11366', '11367', '11368', '11369', '11370', '11372', '11373', '11374', '11375', '11377', '11378', '11379', '11385', '11411', '11412', '11413', '11414', '11415', '11416', '11417', '11418', '11419', '11420', '11421', '11422', '11423', '11426', '11427', '11428', '11429', '11432', '11433', '11434', '11435', '11436', '11507', '11510', '11514', '11516', '11518', '11520', '11530', '11542', '11545', '11550', '11552', '11553', '11554', '11557', '11558', '11559', '11560', '11561', '11563', '11565', '11566', '11570', '11572', '11575', '11576', '11577', '11579', '11580', '11581', '11590', '11598', '11691', '11692', '11701', '11703', '11704', '11706', '11710', '11714', '11717', '11720', '11722', '11725', '11729', '11732', '11735', '11741', '11743', '11746', '11751', '11753', '11754', '11756', '11757', '11758', '11762', '11767', '11771', '11778', '11779', '11780', '11783', '11786', '11787', '11788', '11791', '11793', '11797', '11801', '11803', '11950', '11951', '12018', '12020', '12037', '12043', '12047', '12054', '12061', '12065', '12074', '12077', '12083', '12110', '12118', '12122', '12143', '12157', '12158', '12170', '12180', '12184', '12186', '12188', '12189', '12198', '12203', '12205', '12208', '12209', '12210', '12303', '12309', '12401', '12404', '12428', '12443', '12446', '12466', '12477', '12508', '12518', '12522', '12524', '12525', '12528', '12531', '12533', '12534', '12538', '12540', '12542', '12545', '12549', '12550', '12553', '12561', '12563', '12564', '12566', '12570', '12572', '12580', '12582', '12586', '12589', '12590', '12594', '12601', '12603', '12701', '12721', '12754', '12758', '12771', '12831', '12871', '12883', '12901', '12972', '12983', '13027', '13029', '13039', '13057', '13080', '13090', '13108', '13203', '13204', '13205', '13206', '13207', '13209', '13211', '13212', '13624', '13850', '14009', '14011', '14020', '14032', '14036', '14043', '14051', '14070', '14075', '14086', '14094', '14098', '14103', '14105', '14111', '14125', '14127', '14141', '14150', '14204', '14206', '14207', '14208', '14210', '14211', '14212', '14213', '14214', '14215', '14216', '14217', '14218', '14219', '14220', '14222', '14223', '14224', '14225', '14226', '14227', '14228', '14304', '14411', '14416', '14428', '14437', '14445', '14450', '14468', '14470', '14472', '14482', '14527', '14530', '14534', '14569', '14580', '14586', '14605', '14606', '14608', '14611', '14612', '14613', '14615', '14616', '14617', '14620', '14621', '14622', '14623', '14626', '14891', '14895', '15012', '15017', '15024', '15025', '15037', '15044', '15045', '15046', '15057', '15068', '15071', '15084', '15090', '15101', '15102', '15108', '15110', '15112', '15116', '15120', '15122', '15126', '15129', '15131', '15132', '15133', '15135', '15136', '15137', '15139', '15140', '15144', '15145', '15146', '15147', '15201', '15202', '15203', '15204', '15205', '15206', '15207', '15208', '15209', '15210', '15211', '15212', '15213', '15214', '15215', '15216', '15217', '15218', '15219', '15220', '15221', '15223', '15224', '15226', '15227', '15228', '15229', '15233', '15234', '15235', '15236', '15237', '15238', '15239', '15241', '15243', '15317', '15330', '15332', '15367', '15401', '15431', '15445', '15522', '15537', '15613', '15644', '15650', '15658', '15666', '15767', '15851', '16046', '16323', '16407', '16428', '16438', '16504', '16507', '16508', '16509', '16510', '16678', '16701', '16801', '16901', '17007', '17011', '17013', '17015', '17018', '17019', '17020', '17022', '17025', '17032', '17033', '17036', '17042', '17046', '17050', '17055', '17057', '17059', '17061', '17065', '17070', '17102', '17103', '17104', '17109', '17110', '17111', '17112', '17113', '17201', '17202', '17222', '17225', '17233', '17236', '17241', '17257', '17268', '17301', '17313', '17314', '17315', '17319', '17322', '17324', '17325', '17327', '17331', '17339', '17340', '17345', '17347', '17349', '17350', '17356', '17360', '17361', '17362', '17363', '17366', '17368', '17370', '17401', '17402', '17403', '17404', '17406', '17408', '17501', '17512', '17517', '17519', '17520', '17522', '17538', '17540', '17543', '17545', '17547', '17551', '17552', '17554', '17555', '17557', '17560', '17566', '17569', '17578', '17579', '17584', '17601', '17602', '17603', '17821', '17837', '18011', '18013', '18014', '18015', '18017', '18018', '18020', '18031', '18032', '18034', '18036', '18037', '18038', '18040', '18041', '18042', '18045', '18049', '18052', '18054', '18055', '18058', '18062', '18064', '18066', '18067', '18071', '18072', '18073', '18078', '18080', '18088', '18091', '18092', '18102', '18103', '18104', '18106', '18109', '18202', '18210', '18232', '18235', '18255', '18301', '18302', '18324', '18325', '18330', '18336', '18337', '18343', '18353', '18360', '18424', '18426', '18428', '18458', '18466', '18901', '18902', '18914', '18938', '18940', '18942', '18944', '18947', '18951', '18960', '18964', '18966', '18969', '18972', '18974', '18976', '19001', '19002', '19003', '19004', '19006', '19007', '19010', '19012', '19013', '19014', '19015', '19018', '19020', '19021', '19023', '19025', '19026', '19027', '19029', '19030', '19031', '19033', '19034', '19036', '19038', '19040', '19041', '19044', '19046', '19047', '19050', '19054', '19055', '19056', '19057', '19061', '19063', '19064', '19067', '19070', '19072', '19074', '19075', '19076', '19078', '19079', '19082', '19083', '19086', '19090', '19094', '19095', '19096', '19102', '19103', '19104', '19106', '19107', '19111', '19114', '19115', '19116', '19118', '19119', '19120', '19121', '19122', '19123', '19124', '19125', '19126', '19127', '19128', '19129', '19130', '19131', '19132', '19133', '19134', '19135', '19136', '19137', '19138', '19139', '19140', '19141', '19142', '19143', '19144', '19145', '19146', '19147', '19148', '19149', '19150', '19151', '19153', '19154', '19301', '19311', '19312', '19317', '19320', '19330', '19333', '19335', '19341', '19343', '19344', '19348', '19350', '19355', '19362', '19363', '19365', '19380', '19382', '19390', '19401', '19403', '19405', '19406', '19422', '19425', '19426', '19428', '19438', '19440', '19444', '19446', '19454', '19460', '19462', '19464', '19465', '19468', '19473', '19475', '19504', '19505', '19506', '19507', '19508', '19512', '19518', '19520', '19522', '19525', '19526', '19530', '19533', '19539', '19540', '19541', '19543', '19547', '19551', '19555', '19560', '19565', '19567', '19601', '19602', '19605', '19606', '19607', '19608', '19609', '19610', '19611', '19701', '19702', '19703', '19709', '19711', '19713', '19720', '19734', '19801', '19802', '19803', '19804', '19805', '19806', '19808', '19809', '19810', '19901', '19904', '19934', '19938', '19943', '19945', '19946', '19950', '19952', '19953', '19958', '19962', '19963', '19968', '19975', '19977', '20001', '20002', '20003', '20005', '20007', '20008', '20009', '20010', '20011', '20012', '20015', '20016', '20017', '20018', '20019', '20020', '20024', '20032', '20036', '20037', '20105', '20106', '20109', '20110', '20111', '20112', '20115', '20119', '20120', '20121', '20132', '20136', '20141', '20147', '20148', '20151', '20152', '20155', '20164', '20165', '20166', '20169', '20170', '20171', '20175', '20176', '20180', '20181', '20186', '20187', '20190', '20191', '20601', '20602', '20603', '20619', '20634', '20636', '20639', '20640', '20646', '20650', '20653', '20657', '20659', '20664', '20676', '20678', '20685', '20695', '20705', '20706', '20707', '20708', '20710', '20712', '20714', '20715', '20716', '20722', '20723', '20724', '20732', '20735', '20736', '20737', '20740', '20743', '20744', '20745', '20746', '20747', '20748', '20754', '20764', '20770', '20772', '20774', '20781', '20782', '20783', '20784', '20785', '20794', '20814', '20815', '20817', '20832', '20833', '20850', '20851', '20852', '20853', '20854', '20855', '20866', '20872', '20874', '20876', '20877', '20878', '20879', '20886', '20901', '20902', '20903', '20904', '20905', '20906', '20910', '21001', '21009', '21012', '21014', '21015', '21017', '21028', '21029', '21037', '21040', '21042', '21043', '21044', '21045', '21046', '21050', '21054', '21060', '21061', '21074', '21075', '21076', '21078', '21084', '21085', '21090', '21093', '21102', '21108', '21113', '21114', '21117', '21122', '21128', '21133', '21136', '21144', '21146', '21152', '21157', '21158', '21161', '21163', '21201', '21202', '21204', '21205', '21206', '21207', '21208', '21209', '21211', '21212', '21213', '21214', '21215', '21216', '21217', '21218', '21219', '21220', '21221', '21222', '21223', '21224', '21225', '21227', '21228', '21229', '21230', '21234', '21236', '21237', '21239', '21244', '21286', '21401', '21403', '21409', '21601', '21619', '21620', '21629', '21631', '21632', '21639', '21651', '21655', '21658', '21661', '21663', '21666', '21673', '21701', '21702', '21703', '21704', '21713', '21716', '21722', '21727', '21740', '21742', '21750', '21754', '21755', '21756', '21758', '21769', '21770', '21771', '21773', '21774', '21776', '21783', '21784', '21788', '21791', '21793', '21795', '21801', '21804', '21817', '21853', '21875', '21901', '21903', '21904', '21911', '21915', '21918', '21921', '22003', '22015', '22025', '22026', '22030', '22031', '22032', '22039', '22041', '22042', '22043', '22044', '22046', '22079', '22102', '22124', '22150', '22151', '22152', '22153', '22172', '22180', '22181', '22182', '22191', '22192', '22193', '22201', '22202', '22203', '22204', '22205', '22206', '22207', '22209', '22302', '22303', '22304', '22306', '22309', '22310', '22312', '22314', '22315', '22401', '22405', '22406', '22407', '22408', '22485', '22503', '22508', '22546', '22551', '22553', '22554', '22556', '22560', '22572', '22580', '22601', '22602', '22603', '22611', '22630', '22642', '22645', '22655', '22701', '22712', '22727', '22728', '22801', '22802', '22827', '22835', '22849', '22851', '22853', '22901', '22902', '22903', '22911', '22923', '22932', '22936', '22960', '22963', '22967', '22968', '22973', '22974', '23002', '23005', '23009', '23030', '23040', '23060', '23061', '23063', '23072', '23075', '23086', '23103', '23111', '23112', '23113', '23114', '23116', '23124', '23139', '23140', '23141', '23150', '23168', '23181', '23185', '23188', '23220', '23221', '23222', '23223', '23224', '23225', '23226', '23227', '23228', '23229', '23230', '23231', '23233', '23234', '23235', '23236', '23237', '23238', '23294', '23310', '23314', '23320', '23321', '23322', '23323', '23324', '23325', '23430', '23434', '23435', '23451', '23452', '23453', '23454', '23455', '23456', '23457', '23462', '23464', '23502', '23503', '23504', '23505', '23508', '23509', '23510', '23513', '23517', '23518', '23601', '23602', '23605', '23606', '23607', '23608', '23661', '23662', '23663', '23664', '23666', '23669', '23690', '23693', '23701', '23702', '23703', '23704', '23707', '23831', '23832', '23836', '23837', '23851', '23860', '23875', '23885', '23901', '24012', '24014', '24016', '24018', '24019', '24054', '24064', '24073', '24112', '24153', '24179', '24201', '24251', '24330', '24333', '24348', '24416', '24422', '24426', '24450', '24501', '24502', '24503', '24504', '24531', '24549', '24551', '24557', '24590', '24605', '24630', '24651', '24954', '24986', '25064', '25130', '25143', '25177', '25276', '25302', '25401', '25403', '25404', '25405', '25411', '25413', '25414', '25419', '25425', '25427', '25428', '25430', '25443', '25504', '25508', '25510', '25526', '25541', '25550', '25570', '25601', '25703', '25705', '26003', '26037', '26038', '26062', '26070', '26101', '26104', '26143', '26155', '26159', '26164', '26201', '26241', '26250', '26301', '26354', '26416', '26426', '26452', '26505', '26537', '26554', '26582', '26704', '26711', '26726', '26753', '26757', '26764', '26847', '27012', '27017', '27021', '27023', '27030', '27040', '27041', '27043', '27052', '27101', '27103', '27104', '27107', '27127', '27203', '27215', '27217', '27244', '27253', '27260', '27262', '27265', '27278', '27282', '27283', '27284', '27292', '27295', '27302', '27306', '27312', '27330', '27332', '27349', '27360', '27371', '27377', '27379', '27401', '27403', '27405', '27406', '27407', '27408', '27409', '27410', '27455', '27501', '27502', '27504', '27505', '27509', '27510', '27511', '27513', '27514', '27516', '27517', '27518', '27519', '27520', '27521', '27523', '27524', '27525', '27526', '27527', '27529', '27530', '27534', '27537', '27540', '27542', '27546', '27557', '27560', '27569', '27571', '27573', '27574', '27576', '27577', '27583', '27587', '27591', '27596', '27597', '27601', '27603', '27604', '27606', '27607', '27609', '27610', '27612', '27613', '27614', '27615', '27616', '27703', '27704', '27705', '27707', '27712', '27713', '27803', '27804', '27807', '27828', '27834', '27856', '27858', '27863', '27882', '27889', '27892', '27909', '27921', '27932', '27948', '27949', '27954', '27958', '28012', '28016', '28021', '28023', '28025', '28027', '28031', '28032', '28034', '28036', '28037', '28040', '28043', '28052', '28054', '28056', '28075', '28078', '28079', '28081', '28083', '28088', '28092', '28098', '28103', '28104', '28105', '28107', '28110', '28112', '28114', '28115', '28117', '28124', '28134', '28138', '28139', '28144', '28146', '28147', '28164', '28166', '28170', '28173', '28174', '28202', '28203', '28204', '28205', '28206', '28208', '28209', '28210', '28211', '28212', '28213', '28214', '28215', '28216', '28217', '28226', '28227', '28262', '28269', '28270', '28273', '28277', '28278', '28301', '28303', '28304', '28306', '28311', '28312', '28314', '28315', '28323', '28326', '28327', '28328', '28333', '28334', '28339', '28340', '28348', '28356', '28358', '28364', '28365', '28371', '28376', '28377', '28384', '28390', '28391', '28394', '28398', '28401', '28403', '28405', '28409', '28411', '28412', '28422', '28428', '28431', '28443', '28445', '28451', '28457', '28458', '28460', '28461', '28462', '28463', '28465', '28466', '28467', '28468', '28470', '28472', '28478', '28479', '28512', '28513', '28516', '28518', '28532', '28539', '28540', '28546', '28555', '28557', '28560', '28570', '28572', '28574', '28580', '28584', '28586', '28590', '28594', '28601', '28602', '28604', '28607', '28610', '28613', '28621', '28625', '28634', '28650', '28658', '28673', '28675', '28677', '28678', '28681', '28692', '28694', '28704', '28705', '28711', '28713', '28714', '28715', '28721', '28731', '28732', '28739', '28746', '28748', '28752', '28753', '28754', '28759', '28761', '28762', '28771', '28777', '28782', '28785', '28786', '28787', '28790', '28791', '28792', '28801', '28803', '28804', '28805', '28806', '29006', '29016', '29020', '29033', '29036', '29040', '29044', '29045', '29063', '29072', '29073', '29102', '29127', '29138', '29148', '29160', '29169', '29170', '29201', '29203', '29204', '29205', '29206', '29209', '29210', '29212', '29223', '29229', '29301', '29303', '29306', '29307', '29323', '29330', '29334', '29349', '29372', '29379', '29388', '29401', '29403', '29405', '29406', '29407', '29410', '29412', '29414', '29418', '29420', '29445', '29451', '29456', '29461', '29464', '29466', '29472', '29477', '29483', '29485', '29488', '29492', '29501', '29511', '29526', '29527', '29532', '29540', '29541', '29550', '29554', '29556', '29566', '29568', '29569', '29575', '29576', '29577', '29579', '29582', '29585', '29588', '29601', '29605', '29607', '29609', '29611', '29615', '29621', '29626', '29642', '29662', '29669', '29673', '29681', '29687', '29696', '29707', '29732', '29801', '29803', '29824', '29829', '29832', '29835', '29841', '29851', '29860', '29902', '29906', '29907', '29909', '29910', '29920', '29926', '30002', '30004', '30005', '30008', '30009', '30011', '30012', '30013', '30014', '30016', '30017', '30019', '30021', '30022', '30024', '30025', '30028', '30030', '30032', '30033', '30034', '30035', '30038', '30039', '30040', '30041', '30043', '30044', '30045', '30046', '30047', '30052', '30054', '30058', '30060', '30062', '30064', '30066', '30067', '30068', '30071', '30075', '30076', '30078', '30080', '30082', '30083', '30084', '30087', '30088', '30092', '30093', '30094', '30096', '30097', '30101', '30102', '30103', '30106', '30107', '30108', '30110', '30113', '30114', '30115', '30116', '30117', '30120', '30121', '30122', '30125', '30126', '30127', '30134', '30135', '30139', '30141', '30143', '30144', '30145', '30152', '30153', '30168', '30171', '30175', '30179', '30180', '30183', '30184', '30187', '30188', '30189', '30204', '30214', '30215', '30217', '30222', '30223', '30224', '30228', '30230', '30233', '30236', '30238', '30240', '30241', '30248', '30252', '30253', '30257', '30259', '30260', '30265', '30269', '30273', '30274', '30276', '30277', '30281', '30286', '30288', '30292', '30294', '30295', '30296', '30297', '30305', '30306', '30307', '30308', '30309', '30310', '30311', '30312', '30313', '30314', '30315', '30316', '30317', '30318', '30319', '30324', '30326', '30328', '30329', '30331', '30337', '30338', '30339', '30340', '30341', '30342', '30344', '30345', '30349', '30350', '30354', '30360', '30401', '30417', '30427', '30436', '30439', '30457', '30458', '30461', '30467', '30474', '30501', '30504', '30506', '30507', '30512', '30513', '30517', '30518', '30519', '30521', '30523', '30525', '30527', '30528', '30530', '30535', '30536', '30540', '30542', '30543', '30546', '30548', '30549', '30553', '30554', '30557', '30558', '30559', '30560', '30563', '30564', '30566', '30571', '30577', '30582', '30601', '30605', '30620', '30622', '30625', '30628', '30633', '30635', '30642', '30643', '30650', '30655', '30656', '30662', '30666', '30673', '30680', '30701', '30705', '30707', '30735', '30736', '30738', '30741', '30747', '30750', '30752', '30753', '30802', '30809', '30813', '30814', '30817', '30824', '30830', '30904', '30907', '31005', '31008', '31014', '31015', '31021', '31023', '31024', '31029', '31031', '31032', '31036', '31052', '31064', '31069', '31082', '31087', '31088', '31093', '31201', '31204', '31206', '31210', '31211', '31216', '31217', '31302', '31305', '31308', '31312', '31313', '31320', '31322', '31324', '31326', '31329', '31331', '31401', '31404', '31405', '31406', '31407', '31408', '31410', '31411', '31415', '31419', '31510', '31513', '31516', '31520', '31533', '31535', '31537', '31539', '31545', '31546', '31548', '31558', '31569', '31601', '31602', '31605', '31620', '31632', '31635', '31636', '31639', '31643', '31709', '31714', '31730', '31750', '31768', '31779', '31788', '31794', '31811', '31822', '31831', '31833', '31903', '31904', '31906', '31907', '31909', '32003', '32011', '32024', '32034', '32040', '32043', '32046', '32052', '32054', '32060', '32063', '32065', '32068', '32073', '32080', '32082', '32084', '32086', '32092', '32097', '32110', '32113', '32114', '32117', '32118', '32119', '32127', '32128', '32129', '32130', '32134', '32137', '32141', '32164', '32168', '32174', '32176', '32179', '32205', '32206', '32207', '32208', '32209', '32210', '32211', '32216', '32217', '32218', '32219', '32220', '32221', '32223', '32224', '32225', '32226', '32233', '32244', '32246', '32250', '32254', '32256', '32257', '32258', '32259', '32266', '32277', '32301', '32303', '32304', '32305', '32308', '32309', '32310', '32311', '32312', '32317', '32321', '32324', '32340', '32344', '32424', '32425', '32428', '32439', '32456', '32501', '32503', '32504', '32505', '32506', '32507', '32514', '32526', '32533', '32535', '32536', '32539', '32541', '32547', '32548', '32561', '32563', '32565', '32566', '32569', '32570', '32571', '32578', '32579', '32580', '32583', '32601', '32605', '32606', '32607', '32608', '32615', '32617', '32626', '32641', '32643', '32653', '32656', '32667', '32669', '32680', '32686', '32693', '32696', '32701', '32703', '32707', '32708', '32712', '32713', '32714', '32720', '32724', '32725', '32726', '32730', '32736', '32738', '32744', '32746', '32750', '32751', '32754', '32757', '32763', '32765', '32766', '32771', '32773', '32778', '32779', '32780', '32784', '32789', '32792', '32796', '32801', '32803', '32804', '32805', '32806', '32807', '32808', '32809', '32810', '32811', '32812', '32814', '32817', '32818', '32819', '32821', '32822', '32824', '32825', '32826', '32827', '32828', '32829', '32835', '32836', '32837', '32839', '32901', '32903', '32904', '32905', '32907', '32908', '32909', '32920', '32922', '32926', '32927', '32931', '32934', '32935', '32937', '32940', '32950', '32951', '32952', '32953', '32955', '32958', '32960', '32962', '32966', '32967', '32968', '32976', '33004', '33009', '33010', '33012', '33013', '33014', '33015', '33016', '33018', '33019', '33020', '33021', '33023', '33024', '33025', '33026', '33027', '33028', '33029', '33030', '33031', '33032', '33033', '33034', '33035', '33043', '33054', '33055', '33056', '33060', '33062', '33063', '33064', '33065', '33066', '33067', '33068', '33069', '33071', '33073', '33125', '33126', '33127', '33129', '33130', '33131', '33132', '33133', '33134', '33135', '33136', '33137', '33138', '33139', '33140', '33141', '33142', '33143', '33144', '33145', '33146', '33147', '33150', '33154', '33155', '33156', '33157', '33160', '33161', '33165', '33166', '33167', '33168', '33169', '33170', '33172', '33173', '33174', '33175', '33176', '33177', '33178', '33179', '33180', '33181', '33183', '33184', '33185', '33186', '33189', '33190', '33196', '33301', '33304', '33305', '33308', '33309', '33311', '33312', '33313', '33314', '33315', '33316', '33317', '33319', '33321', '33322', '33323', '33324', '33325', '33326', '33327', '33328', '33331', '33334', '33351', '33401', '33403', '33404', '33405', '33406', '33407', '33408', '33409', '33410', '33411', '33412', '33413', '33415', '33417', '33418', '33426', '33428', '33431', '33432', '33433', '33434', '33435', '33436', '33437', '33440', '33441', '33442', '33444', '33445', '33446', '33449', '33455', '33458', '33460', '33461', '33462', '33463', '33467', '33472', '33476', '33478', '33483', '33484', '33486', '33487', '33496', '33498', '33510', '33511', '33523', '33525', '33534', '33540', '33541', '33542', '33543', '33544', '33545', '33549', '33556', '33558', '33559', '33563', '33565', '33566', '33567', '33569', '33570', '33572', '33573', '33576', '33578', '33579', '33584', '33592', '33594', '33596', '33597', '33598', '33602', '33603', '33604', '33605', '33606', '33607', '33609', '33610', '33611', '33612', '33613', '33614', '33615', '33616', '33617', '33618', '33619', '33624', '33625', '33626', '33629', '33634', '33635', '33637', '33647', '33701', '33702', '33703', '33704', '33705', '33706', '33707', '33708', '33709', '33710', '33711', '33712', '33713', '33714', '33715', '33716', '33755', '33756', '33759', '33760', '33761', '33762', '33763', '33764', '33765', '33767', '33770', '33771', '33772', '33773', '33774', '33776', '33777', '33778', '33781', '33782', '33785', '33801', '33803', '33805', '33809', '33810', '33811', '33812', '33813', '33815', '33823', '33825', '33830', '33837', '33839', '33841', '33843', '33844', '33850', '33853', '33859', '33860', '33868', '33870', '33873', '33875', '33880', '33881', '33884', '33896', '33897', '33898', '33901', '33903', '33904', '33905', '33907', '33908', '33909', '33912', '33913', '33914', '33916', '33917', '33919', '33920', '33928', '33935', '33936', '33952', '33957', '33966', '33967', '33971', '33972', '33973', '33974', '33976', '33980', '33981', '33982', '33983', '33990', '33991', '33993', '34135', '34202', '34203', '34205', '34207', '34208', '34209', '34211', '34212', '34219', '34221', '34222', '34231', '34232', '34233', '34234', '34235', '34236', '34237', '34238', '34239', '34240', '34242', '34243', '34266', '34286', '34287', '34420', '34431', '34434', '34442', '34450', '34452', '34453', '34461', '34465', '34470', '34471', '34472', '34473', '34474', '34475', '34476', '34479', '34480', '34481', '34482', '34488', '34491', '34601', '34602', '34606', '34607', '34608', '34609', '34610', '34613', '34614', '34637', '34638', '34639', '34652', '34653', '34654', '34655', '34667', '34668', '34669', '34677', '34683', '34684', '34685', '34689', '34691', '34695', '34698', '34711', '34714', '34715', '34731', '34736', '34737', '34741', '34743', '34744', '34746', '34747', '34748', '34756', '34758', '34759', '34761', '34769', '34771', '34772', '34787', '34945', '34946', '34947', '34949', '34950', '34951', '34952', '34953', '34957', '34972', '34982', '34983', '34984', '34986', '34987', '34990', '34994', '34997', '35004', '35005', '35007', '35010', '35016', '35023', '35031', '35045', '35046', '35051', '35055', '35064', '35071', '35079', '35080', '35094', '35117', '35118', '35121', '35124', '35125', '35126', '35127', '35128', '35146', '35151', '35160', '35175', '35179', '35180', '35204', '35205', '35206', '35207', '35208', '35210', '35211', '35212', '35214', '35215', '35216', '35217', '35218', '35222', '35224', '35226', '35228', '35235', '35242', '35243', '35244', '35401', '35404', '35405', '35452', '35453', '35473', '35501', '35503', '35504', '35550', '35555', '35565', '35570', '35578', '35580', '35581', '35594', '35619', '35640', '35650', '35653', '35654', '35660', '35661', '35672', '35673', '35749', '35750', '35758', '35759', '35761', '35768', '35769', '35773', '35801', '35802', '35803', '35805', '35810', '35811', '35816', '35905', '35906', '35951', '35952', '35957', '35959', '35960', '35962', '35967', '35968', '35971', '35976', '35978', '35980', '35986', '36022', '36024', '36025', '36027', '36049', '36054', '36064', '36066', '36067', '36078', '36079', '36081', '36092', '36105', '36109', '36111', '36116', '36117', '36201', '36203', '36206', '36207', '36251', '36264', '36265', '36271', '36274', '36277', '36301', '36303', '36305', '36310', '36322', '36330', '36344', '36345', '36351', '36352', '36360', '36375', '36420', '36421', '36426', '36467', '36477', '36502', '36507', '36522', '36526', '36527', '36530', '36532', '36535', '36541', '36542', '36544', '36549', '36551', '36567', '36571', '36575', '36576', '36580', '36582', '36587', '36603', '36604', '36605', '36606', '36608', '36609', '36610', '36611', '36613', '36617', '36618', '36619', '36693', '36695', '36701', '36703', '36804', '36830', '36853', '36854', '36861', '36862', '36863', '36867', '36869', '36870', '36874', '36877', '37013', '37015', '37020', '37022', '37025', '37027', '37029', '37030', '37032', '37033', '37034', '37037', '37040', '37042', '37043', '37048', '37049', '37055', '37058', '37061', '37062', '37064', '37066', '37067', '37069', '37072', '37073', '37074', '37075', '37076', '37083', '37086', '37091', '37096', '37098', '37101', '37115', '37122', '37127', '37128', '37129', '37130', '37134', '37138', '37143', '37144', '37146', '37148', '37150', '37160', '37166', '37167', '37172', '37174', '37179', '37183', '37184', '37185', '37187', '37188', '37189', '37190', '37203', '37204', '37205', '37206', '37207', '37208', '37209', '37210', '37211', '37212', '37214', '37216', '37217', '37218', '37221', '37307', '37321', '37322', '37324', '37327', '37330', '37336', '37341', '37342', '37343', '37352', '37354', '37355', '37356', '37363', '37367', '37380', '37381', '37385', '37387', '37388', '37397', '37398', '37404', '37405', '37406', '37411', '37415', '37421', '37601', '37615', '37617', '37620', '37640', '37642', '37650', '37659', '37660', '37683', '37686', '37701', '37705', '37708', '37709', '37711', '37721', '37722', '37737', '37742', '37757', '37760', '37764', '37766', '37777', '37801', '37803', '37804', '37807', '37811', '37818', '37820', '37821', '37840', '37843', '37849', '37853', '37854', '37861', '37865', '37871', '37873', '37874', '37876', '37880', '37882', '37886', '37887', '37890', '37909', '37912', '37914', '37917', '37918', '37919', '37920', '37921', '37922', '37923', '37931', '37932', '37934', '37938', '38001', '38002', '38006', '38008', '38012', '38016', '38017', '38018', '38019', '38023', '38049', '38052', '38053', '38057', '38058', '38063', '38075', '38103', '38104', '38106', '38107', '38108', '38109', '38111', '38112', '38114', '38115', '38116', '38117', '38118', '38119', '38120', '38122', '38125', '38126', '38127', '38128', '38133', '38134', '38135', '38138', '38139', '38141', '38201', '38221', '38233', '38242', '38260', '38261', '38310', '38315', '38320', '38326', '38329', '38330', '38340', '38343', '38344', '38351', '38355', '38358', '38363', '38367', '38372', '38375', '38382', '38449', '38451', '38456', '38462', '38464', '38468', '38469', '38478', '38483', '38485', '38556', '38562', '38570', '38574', '38583', '38585', '38606', '38614', '38618', '38637', '38821', '38834', '38851', '38852', '38860', '38901', '38930', '38965', '38967', '39047', '39056', '39059', '39074', '39083', '39117', '39154', '39170', '39180', '39183', '39204', '39206', '39209', '39211', '39212', '39213', '39272', '39341', '39401', '39426', '39428', '39452', '39465', '39466', '39470', '39474', '39475', '39479', '39501', '39503', '39507', '39520', '39525', '39530', '39531', '39532', '39553', '39556', '39560', '39562', '39563', '39564', '39565', '39567', '39571', '39574', '39576', '39581', '39648', '39652', '39773', '39817', '39819', '39823', '39828', '39837', '39842', '39845', '40004', '40013', '40014', '40033', '40056', '40059', '40065', '40069', '40071', '40118', '40160', '40162', '40165', '40175', '40203', '40204', '40205', '40206', '40207', '40208', '40210', '40212', '40213', '40214', '40215', '40216', '40217', '40218', '40219', '40220', '40222', '40223', '40228', '40229', '40242', '40243', '40245', '40258', '40272', '40291', '40299', '40324', '40330', '40336', '40353', '40356', '40360', '40361', '40380', '40391', '40403', '40422', '40444', '40475', '40484', '40502', '40503', '40504', '40505', '40508', '40509', '40511', '40513', '40514', '40515', '40517', '40701', '40741', '40744', '41005', '41008', '41011', '41014', '41015', '41016', '41017', '41018', '41030', '41035', '41040', '41042', '41048', '41075', '41076', '41094', '41101', '41102', '41129', '41139', '41143', '41164', '41175', '41179', '41501', '41653', '42001', '42003', '42064', '42101', '42103', '42129', '42164', '42206', '42240', '42261', '42276', '42301', '42303', '42345', '42366', '42408', '42420', '42431', '42501', '42539', '42567', '42602', '42633', '42642', '42701', '42718', '42728', '42743', '42748', '43004', '43011', '43015', '43016', '43017', '43019', '43021', '43023', '43025', '43026', '43028', '43031', '43035', '43040', '43044', '43046', '43050', '43054', '43055', '43056', '43062', '43065', '43068', '43072', '43074', '43078', '43081', '43082', '43085', '43102', '43112', '43119', '43123', '43125', '43130', '43147', '43160', '43201', '43202', '43203', '43204', '43205', '43206', '43207', '43209', '43211', '43212', '43213', '43214', '43215', '43219', '43220', '43221', '43223', '43224', '43227', '43228', '43229', '43230', '43231', '43232', '43235', '43302', '43310', '43311', '43315', '43316', '43324', '43326', '43331', '43334', '43338', '43342', '43348', '43351', '43357', '43410', '43412', '43416', '43420', '43430', '43431', '43440', '43449', '43452', '43469', '43502', '43506', '43512', '43515', '43526', '43532', '43537', '43543', '43545', '43558', '43567', '43604', '43605', '43606', '43607', '43608', '43609', '43611', '43612', '43613', '43614', '43615', '43616', '43623', '43701', '43731', '43764', '43793', '43812', '43822', '43832', '43845', '43952', '43953', '44001', '44011', '44012', '44017', '44022', '44028', '44035', '44039', '44044', '44050', '44052', '44055', '44057', '44060', '44062', '44065', '44067', '44070', '44074', '44077', '44081', '44087', '44090', '44092', '44094', '44095', '44102', '44103', '44104', '44105', '44107', '44108', '44109', '44110', '44111', '44112', '44113', '44116', '44117', '44118', '44119', '44120', '44121', '44122', '44123', '44124', '44125', '44126', '44127', '44128', '44129', '44130', '44131', '44132', '44133', '44134', '44135', '44136', '44137', '44138', '44139', '44140', '44142', '44143', '44144', '44145', '44146', '44147', '44149', '44201', '44203', '44212', '44215', '44216', '44221', '44223', '44224', '44236', '44241', '44253', '44254', '44256', '44260', '44262', '44266', '44273', '44275', '44278', '44281', '44286', '44287', '44301', '44302', '44303', '44305', '44306', '44307', '44310', '44311', '44312', '44313', '44314', '44319', '44320', '44321', '44333', '44420', '44438', '44444', '44471', '44504', '44511', '44512', '44514', '44515', '44609', '44612', '44615', '44622', '44626', '44641', '44644', '44646', '44647', '44656', '44657', '44662', '44663', '44672', '44676', '44680', '44683', '44685', '44703', '44705', '44706', '44707', '44708', '44709', '44710', '44714', '44718', '44720', '44721', '44730', '44827', '44830', '44833', '44870', '44883', '45002', '45005', '45011', '45013', '45014', '45015', '45030', '45036', '45039', '45040', '45042', '45044', '45050', '45065', '45066', '45067', '45068', '45069', '45102', '45103', '45121', '45123', '45133', '45140', '45152', '45154', '45171', '45176', '45177', '45204', '45205', '45206', '45207', '45208', '45211', '45212', '45213', '45214', '45215', '45216', '45219', '45220', '45223', '45224', '45225', '45226', '45227', '45229', '45230', '45231', '45233', '45236', '45237', '45238', '45239', '45240', '45242', '45244', '45245', '45246', '45247', '45248', '45249', '45251', '45252', '45255', '45309', '45311', '45320', '45322', '45324', '45327', '45335', '45338', '45342', '45344', '45347', '45365', '45377', '45381', '45402', '45403', '45405', '45406', '45409', '45410', '45414', '45415', '45416', '45417', '45419', '45420', '45424', '45426', '45429', '45431', '45434', '45439', '45449', '45458', '45503', '45504', '45505', '45506', '45631', '45638', '45640', '45648', '45660', '45663', '45690', '45692', '45693', '45694', '45801', '45804', '45805', '45810', '45822', '45828', '45830', '45833', '45865', '45869', '45875', '45879', '45885', '45891', '45895', '46001', '46011', '46012', '46013', '46016', '46017', '46032', '46033', '46034', '46037', '46038', '46040', '46051', '46055', '46060', '46062', '46072', '46074', '46077', '46106', '46107', '46112', '46113', '46118', '46120', '46121', '46122', '46123', '46124', '46126', '46131', '46135', '46140', '46142', '46143', '46160', '46163', '46164', '46167', '46168', '46173', '46176', '46181', '46184', '46201', '46202', '46203', '46205', '46208', '46214', '46217', '46218', '46219', '46220', '46221', '46222', '46224', '46225', '46226', '46227', '46228', '46229', '46231', '46234', '46235', '46236', '46237', '46239', '46240', '46241', '46250', '46254', '46260', '46268', '46278', '46280', '46303', '46307', '46310', '46311', '46312', '46319', '46320', '46321', '46322', '46324', '46327', '46342', '46347', '46360', '46366', '46373', '46375', '46383', '46392', '46394', '46402', '46403', '46404', '46405', '46406', '46407', '46408', '46409', '46507', '46514', '46516', '46526', '46528', '46534', '46544', '46545', '46550', '46552', '46554', '46561', '46565', '46574', '46613', '46614', '46615', '46616', '46619', '46628', '46710', '46711', '46721', '46733', '46738', '46746', '46755', '46761', '46767', '46787', '46795', '46802', '46804', '46805', '46806', '46807', '46808', '46815', '46825', '46835', '46923', '46929', '46932', '46938', '46947', '46953', '46962', '46970', '46975', '46992', '46996', '47006', '47012', '47018', '47022', '47025', '47031', '47032', '47041', '47043', '47060', '47102', '47136', '47164', '47165', '47167', '47170', '47220', '47240', '47243', '47250', '47265', '47274', '47302', '47303', '47327', '47331', '47336', '47348', '47353', '47356', '47371', '47390', '47394', '47401', '47403', '47408', '47421', '47424', '47432', '47433', '47438', '47441', '47446', '47448', '47454', '47460', '47462', '47501', '47512', '47553', '47567', '47579', '47586', '47591', '47610', '47620', '47635', '47638', '47639', '47648', '47660', '47665', '47670', '47713', '47714', '47715', '47834', '47842', '47872', '47882', '47904', '47905', '47906', '47909', '47918', '47923', '47932', '47944', '47959', '47960', '47978', '47987', '47993', '48006', '48009', '48015', '48017', '48021', '48025', '48026', '48033', '48034', '48035', '48038', '48042', '48043', '48044', '48045', '48047', '48051', '48060', '48065', '48066', '48067', '48069', '48070', '48071', '48072', '48073', '48075', '48076', '48080', '48081', '48082', '48083', '48084', '48085', '48088', '48089', '48091', '48092', '48093', '48094', '48095', '48097', '48098', '48101', '48104', '48108', '48111', '48114', '48116', '48118', '48122', '48124', '48125', '48127', '48128', '48130', '48134', '48135', '48137', '48138', '48141', '48146', '48150', '48152', '48154', '48158', '48160', '48164', '48165', '48167', '48168', '48173', '48174', '48176', '48178', '48179', '48180', '48182', '48183', '48184', '48185', '48186', '48187', '48189', '48192', '48193', '48195', '48197', '48198', '48202', '48203', '48204', '48205', '48206', '48207', '48208', '48209', '48210', '48211', '48212', '48213', '48214', '48215', '48217', '48218', '48219', '48220', '48221', '48223', '48224', '48225', '48227', '48228', '48229', '48234', '48235', '48237', '48238', '48239', '48240', '48301', '48302', '48304', '48306', '48307', '48309', '48310', '48312', '48313', '48314', '48315', '48316', '48317', '48320', '48322', '48323', '48324', '48326', '48327', '48328', '48329', '48334', '48335', '48336', '48340', '48341', '48342', '48346', '48348', '48350', '48353', '48356', '48357', '48359', '48360', '48362', '48371', '48377', '48381', '48382', '48383', '48386', '48390', '48393', '48420', '48439', '48442', '48446', '48462', '48503', '48504', '48505', '48507', '48519', '48529', '48603', '48624', '48625', '48629', '48638', '48653', '48823', '48827', '48836', '48838', '48840', '48843', '48854', '48855', '48858', '48892', '48906', '48910', '48911', '48912', '48915', '49001', '49006', '49007', '49024', '49046', '49047', '49048', '49203', '49285', '49315', '49321', '49330', '49341', '49343', '49345', '49404', '49417', '49418', '49424', '49428', '49437', '49441', '49442', '49444', '49457', '49461', '49464', '49503', '49504', '49505', '49506', '49507', '49508', '49509', '49512', '49519', '49525', '49534', '49544', '49546', '49548', '49601', '49646', '49649', '49686', '49707', '49738', '49740', '49770', '49829', '50009', '50021', '50023', '50035', '50036', '50049', '50109', '50111', '50112', '50125', '50129', '50131', '50138', '50208', '50211', '50212', '50216', '50219', '50237', '50263', '50265', '50266', '50273', '50310', '50312', '50314', '50315', '50316', '50320', '50321', '50322', '50325', '50327', '50401', '50423', '50428', '50438', '50441', '50461', '50588', '50595', '50601', '50616', '50627', '50638', '50644', '50648', '50659', '50662', '50674', '50677', '50701', '50702', '50703', '51040', '51246', '51301', '51351', '51360', '51534', '51537', '51555', '51632', '52003', '52031', '52052', '52057', '52060', '52136', '52172', '52175', '52208', '52211', '52213', '52214', '52240', '52241', '52245', '52246', '52247', '52253', '52301', '52302', '52310', '52314', '52317', '52333', '52338', '52349', '52353', '52358', '52402', '52403', '52404', '52405', '52501', '52531', '52537', '52556', '52577', '52601', '52627', '52632', '52655', '52722', '52748', '52761', '52772', '52803', '52804', '52806', '53005', '53018', '53029', '53045', '53051', '53105', '53110', '53122', '53140', '53142', '53144', '53146', '53151', '53158', '53168', '53179', '53181', '53182', '53186', '53188', '53189', '53202', '53204', '53205', '53206', '53207', '53208', '53209', '53210', '53212', '53213', '53214', '53215', '53216', '53218', '53219', '53220', '53221', '53222', '53223', '53224', '53225', '53228', '53406', '53527', '53528', '53558', '53559', '53562', '53572', '53575', '53589', '53590', '53593', '53703', '53704', '53705', '53711', '53713', '53714', '53716', '53717', '53718', '53719', '54016', '54165', '54166', '54843', '54911', '54913', '54914', '55005', '55006', '55008', '55011', '55013', '55014', '55016', '55025', '55032', '55033', '55037', '55038', '55040', '55043', '55044', '55045', '55047', '55051', '55055', '55056', '55057', '55063', '55068', '55069', '55070', '55071', '55072', '55075', '55076', '55077', '55079', '55082', '55101', '55102', '55103', '55104', '55105', '55106', '55107', '55108', '55109', '55110', '55112', '55113', '55115', '55116', '55117', '55118', '55119', '55121', '55122', '55123', '55124', '55125', '55126', '55128', '55129', '55130', '55303', '55304', '55305', '55306', '55309', '55311', '55313', '55315', '55316', '55317', '55318', '55321', '55322', '55330', '55331', '55337', '55340', '55343', '55344', '55345', '55347', '55349', '55352', '55355', '55358', '55359', '55362', '55364', '55369', '55371', '55372', '55374', '55375', '55378', '55379', '55387', '55388', '55391', '55398', '55401', '55403', '55404', '55405', '55406', '55407', '55408', '55409', '55410', '55411', '55412', '55413', '55414', '55416', '55417', '55418', '55419', '55420', '55421', '55422', '55423', '55425', '55426', '55427', '55428', '55429', '55430', '55431', '55432', '55433', '55434', '55435', '55436', '55437', '55438', '55439', '55441', '55442', '55443', '55447', '55448', '55449', '55614', '55616', '55719', '55741', '55760', '55901', '55902', '55904', '55906', '55920', '55921', '55927', '55944', '55947', '55976', '56001', '56003', '56011', '56057', '56073', '56081', '56085', '56093', '56201', '56308', '56334', '56342', '56345', '56352', '56353', '56358', '56373', '56377', '56431', '56433', '56435', '56443', '56452', '56461', '56467', '56470', '56473', '56474', '56482', '56484', '57106', '57110', '57401', '57701', '57702', '57703', '58078', '58104', '58201', '59011', '59044', '59101', '59102', '59105', '59106', '59404', '59405', '59425', '59601', '59602', '59711', '59714', '59715', '59718', '59722', '59741', '59801', '59802', '59803', '59808', '59847', '59870', '59935', '60002', '60004', '60005', '60007', '60010', '60012', '60013', '60014', '60015', '60016', '60018', '60020', '60022', '60025', '60026', '60030', '60031', '60033', '60040', '60041', '60042', '60044', '60045', '60046', '60047', '60048', '60050', '60051', '60053', '60056', '60060', '60061', '60062', '60064', '60067', '60068', '60069', '60070', '60071', '60073', '60074', '60076', '60077', '60081', '60083', '60084', '60085', '60087', '60089', '60090', '60091', '60093', '60096', '60097', '60098', '60099', '60101', '60102', '60103', '60104', '60106', '60107', '60108', '60110', '60118', '60119', '60120', '60123', '60124', '60126', '60130', '60131', '60133', '60134', '60137', '60139', '60140', '60142', '60143', '60148', '60152', '60153', '60154', '60155', '60160', '60162', '60164', '60169', '60171', '60172', '60173', '60174', '60175', '60176', '60177', '60181', '60185', '60187', '60188', '60189', '60190', '60191', '60192', '60193', '60194', '60195', '60201', '60202', '60203', '60302', '60304', '60305', '60402', '60404', '60406', '60409', '60410', '60411', '60415', '60416', '60419', '60422', '60423', '60425', '60426', '60428', '60429', '60430', '60431', '60432', '60433', '60435', '60436', '60438', '60439', '60440', '60441', '60442', '60443', '60445', '60446', '60447', '60448', '60449', '60452', '60453', '60455', '60457', '60459', '60462', '60463', '60464', '60465', '60466', '60467', '60469', '60472', '60473', '60477', '60478', '60480', '60482', '60484', '60490', '60501', '60502', '60504', '60505', '60506', '60510', '60513', '60514', '60515', '60516', '60517', '60523', '60525', '60526', '60527', '60532', '60534', '60538', '60540', '60542', '60543', '60544', '60545', '60546', '60548', '60552', '60554', '60555', '60558', '60559', '60560', '60561', '60563', '60564', '60565', '60586', '60601', '60605', '60607', '60608', '60609', '60610', '60611', '60612', '60613', '60614', '60615', '60616', '60617', '60618', '60619', '60620', '60621', '60622', '60623', '60624', '60625', '60626', '60628', '60629', '60630', '60631', '60632', '60633', '60634', '60636', '60637', '60638', '60639', '60640', '60641', '60642', '60643', '60644', '60645', '60646', '60647', '60649', '60651', '60652', '60653', '60654', '60655', '60656', '60657', '60659', '60660', '60661', '60706', '60712', '60803', '60804', '60805', '60827', '61010', '61011', '61065', '61072', '61073', '61108', '61109', '61111', '61114', '61115', '61201', '61244', '61264', '61265', '61401', '61448', '61523', '61604', '61614', '61615', '61616', '61801', '61802', '61820', '61821', '61822', '61853', '61951', '62002', '62024', '62025', '62034', '62035', '62040', '62060', '62062', '62067', '62095', '62097', '62201', '62203', '62204', '62205', '62206', '62207', '62208', '62220', '62221', '62223', '62226', '62232', '62234', '62239', '62243', '62246', '62249', '62258', '62260', '62265', '62269', '62275', '62285', '62294', '62401', '62441', '62650', '62839', '63010', '63011', '63012', '63016', '63017', '63019', '63021', '63025', '63026', '63028', '63031', '63033', '63034', '63038', '63042', '63043', '63044', '63048', '63050', '63051', '63052', '63069', '63074', '63084', '63088', '63103', '63104', '63106', '63107', '63109', '63110', '63111', '63112', '63113', '63114', '63115', '63116', '63117', '63118', '63119', '63120', '63121', '63122', '63123', '63125', '63128', '63129', '63130', '63131', '63132', '63133', '63134', '63135', '63136', '63137', '63138', '63139', '63141', '63143', '63144', '63146', '63147', '63304', '63362', '63366', '63368', '63376', '63379', '63385', '63389', '63401', '63601', '63640', '63701', '63775', '63863', '64012', '64014', '64015', '64016', '64029', '64030', '64034', '64050', '64052', '64053', '64055', '64056', '64057', '64058', '64060', '64063', '64064', '64068', '64075', '64081', '64086', '64093', '64105', '64106', '64108', '64109', '64110', '64111', '64112', '64114', '64118', '64119', '64123', '64124', '64126', '64128', '64129', '64130', '64131', '64132', '64133', '64134', '64137', '64138', '64145', '64151', '64154', '64155', '64157', '64601', '64720', '64730', '64747', '64801', '64804', '64834', '64870', '65018', '65020', '65037', '65049', '65079', '65201', '65202', '65203', '65240', '65265', '65270', '65340', '65355', '65459', '65560', '65584', '65613', '65616', '65619', '65622', '65648', '65653', '65672', '65706', '65708', '65737', '65757', '65770', '65775', '65781', '65802', '65807', '65810', '66002', '66007', '66030', '66032', '66040', '66044', '66046', '66047', '66048', '66049', '66053', '66061', '66062', '66064', '66067', '66071', '66085', '66086', '66092', '66101', '66102', '66103', '66104', '66106', '66111', '66112', '66202', '66203', '66204', '66205', '66206', '66207', '66208', '66210', '66212', '66213', '66214', '66215', '66216', '66219', '66221', '66223', '66227', '66434', '66436', '66441', '66502', '66503', '66547', '66604', '66605', '66606', '66614', '66701', '66713', '66720', '66725', '66736', '66739', '66846', '67005', '67013', '67037', '67063', '67068', '67114', '67152', '67156', '67203', '67205', '67208', '67209', '67211', '67212', '67213', '67216', '67217', '67218', '67220', '67235', '67301', '67335', '67337', '67357', '67401', '67410', '67432', '67449', '67456', '67460', '67501', '67502', '67530', '67554', '67601', '67663', '67801', '67846', '67901', '67951', '68003', '68005', '68007', '68022', '68046', '68048', '68066', '68105', '68106', '68107', '68108', '68111', '68114', '68116', '68117', '68122', '68123', '68124', '68127', '68128', '68130', '68131', '68132', '68134', '68136', '68137', '68144', '68154', '68157', '68164', '68310', '68352', '68410', '68434', '68462', '68467', '68503', '68504', '68505', '68506', '68507', '68508', '68510', '68512', '68516', '68521', '68528', '68661', '68701', '68801', '68803', '68818', '68850', '68949', '69361', '70037', '70047', '70070', '70090', '70401', '70403', '70420', '70433', '70435', '70437', '70443', '70445', '70447', '70448', '70452', '70454', '70458', '70460', '70461', '70466', '70471', '70726', '70809', '70810', '70816', '70820', '71112', '71220', '71241', '71601', '71602', '71603', '71635', '71646', '71655', '71667', '71671', '71701', '71730', '71753', '71822', '71852', '71854', '71901', '71913', '71923', '71929', '72002', '72007', '72019', '72023', '72031', '72032', '72086', '72103', '72104', '72110', '72113', '72114', '72116', '72118', '72131', '72143', '72150', '72173', '72176', '72204', '72205', '72206', '72207', '72209', '72211', '72212', '72223', '72301', '72335', '72401', '72432', '72450', '72501', '72529', '72542', '72576', '72601', '72616', '72638', '72701', '72703', '72704', '72712', '72714', '72715', '72719', '72730', '72732', '72734', '72736', '72740', '72744', '72745', '72753', '72756', '72758', '72761', '72762', '72764', '72802', '72834', '72837', '72903', '72908', '72916', '72927', '72933', '72936', '72949', '72956', '73003', '73005', '73008', '73012', '73013', '73020', '73026', '73034', '73045', '73052', '73055', '73064', '73065', '73068', '73069', '73071', '73072', '73075', '73077', '73078', '73084', '73086', '73096', '73098', '73099', '73103', '73105', '73106', '73107', '73108', '73109', '73110', '73111', '73112', '73114', '73115', '73116', '73118', '73119', '73120', '73122', '73127', '73129', '73130', '73132', '73135', '73139', '73159', '73160', '73162', '73165', '73170', '73439', '73446', '73448', '73460', '73501', '73505', '73507', '73533', '73538', '73632', '73662', '73701', '73703', '73750', '73801', '74003', '74006', '74008', '74010', '74011', '74012', '74014', '74015', '74019', '74020', '74033', '74036', '74037', '74044', '74047', '74048', '74053', '74055', '74056', '74063', '74066', '74070', '74075', '74079', '74104', '74105', '74106', '74107', '74110', '74112', '74115', '74127', '74128', '74129', '74133', '74134', '74135', '74136', '74137', '74145', '74146', '74301', '74331', '74344', '74352', '74361', '74401', '74403', '74426', '74432', '74434', '74437', '74441', '74447', '74462', '74464', '74501', '74523', '74525', '74538', '74578', '74653', '74701', '74743', '74745', '74801', '74804', '74820', '74834', '74851', '74855', '74857', '74859', '74864', '74868', '74872', '74873', '74881', '74884', '74948', '74953', '74954', '74959', '74960', '75001', '75002', '75006', '75007', '75009', '75010', '75013', '75019', '75023', '75024', '75025', '75028', '75032', '75034', '75035', '75038', '75039', '75040', '75041', '75042', '75043', '75044', '75050', '75051', '75052', '75056', '75057', '75060', '75061', '75062', '75063', '75065', '75067', '75069', '75070', '75074', '75077', '75080', '75081', '75082', '75087', '75088', '75089', '75093', '75098', '75103', '75104', '75114', '75115', '75116', '75119', '75125', '75126', '75137', '75142', '75146', '75147', '75149', '75152', '75160', '75165', '75167', '75169', '75173', '75180', '75181', '75202', '75203', '75204', '75205', '75206', '75208', '75209', '75211', '75212', '75214', '75215', '75217', '75218', '75219', '75220', '75223', '75225', '75227', '75228', '75231', '75234', '75235', '75237', '75238', '75240', '75241', '75243', '75244', '75248', '75249', '75254', '75287', '75401', '75407', '75409', '75418', '75428', '75440', '75454', '75455', '75482', '75551', '75563', '75602', '75603', '75604', '75633', '75647', '75650', '75652', '75654', '75662', '75670', '75672', '75684', '75686', '75692', '75693', '75701', '75702', '75703', '75704', '75706', '75707', '75708', '75757', '75762', '75773', '75791', '75835', '75840', '75844', '75862', '75935', '75951', '76001', '76002', '76006', '76009', '76010', '76011', '76012', '76013', '76014', '76015', '76016', '76017', '76018', '76020', '76021', '76022', '76031', '76033', '76034', '76036', '76039', '76040', '76043', '76048', '76051', '76053', '76054', '76058', '76060', '76063', '76065', '76078', '76084', '76102', '76103', '76104', '76106', '76107', '76108', '76109', '76110', '76111', '76112', '76114', '76115', '76116', '76117', '76118', '76119', '76120', '76123', '76126', '76131', '76132', '76133', '76134', '76135', '76137', '76140', '76164', '76179', '76180', '76182', '76201', '76205', '76207', '76209', '76210', '76227', '76230', '76234', '76240', '76244', '76248', '76249', '76262', '76308', '76309', '76354', '76365', '76374', '76424', '76442', '76450', '76501', '76502', '76504', '76513', '76520', '76522', '76537', '76541', '76542', '76543', '76548', '76549', '76559', '76574', '76634', '76642', '76643', '76657', '76667', '76706', '76708', '76710', '76711', '76712', '76825', '76901', '76903', '76905', '77002', '77003', '77004', '77005', '77006', '77007', '77009', '77011', '77014', '77015', '77016', '77017', '77018', '77019', '77021', '77024', '77025', '77026', '77027', '77028', '77029', '77030', '77033', '77034', '77035', '77036', '77039', '77042', '77043', '77045', '77048', '77053', '77054', '77055', '77056', '77057', '77058', '77061', '77062', '77063', '77064', '77066', '77069', '77070', '77071', '77072', '77073', '77074', '77075', '77077', '77079', '77080', '77082', '77083', '77084', '77088', '77089', '77090', '77091', '77092', '77093', '77095', '77098', '77099', '77301', '77302', '77304', '77306', '77316', '77318', '77328', '77331', '77338', '77339', '77345', '77346', '77354', '77355', '77356', '77365', '77371', '77372', '77373', '77377', '77378', '77379', '77380', '77381', '77382', '77384', '77385', '77386', '77388', '77401', '77407', '77418', '77423', '77435', '77437', '77441', '77445', '77449', '77450', '77459', '77461', '77469', '77471', '77474', '77477', '77478', '77479', '77488', '77489', '77494', '77498', '77504', '77518', '77520', '77521', '77530', '77536', '77539', '77545', '77546', '77547', '77565', '77568', '77571', '77573', '77584', '77586', '77587', '77590', '77591', '77598', '77619', '77625', '77627', '77640', '77651', '77662', '77701', '77703', '77705', '77706', '77707', '77708', '77801', '77803', '77833', '77836', '77840', '77845', '77879', '77954', '77957', '77964', '77979', '78002', '78006', '78009', '78023', '78028', '78040', '78041', '78043', '78045', '78070', '78073', '78101', '78108', '78109', '78112', '78121', '78124', '78130', '78132', '78133', '78148', '78154', '78155', '78163', '78201', '78203', '78207', '78208', '78209', '78210', '78211', '78212', '78213', '78216', '78217', '78218', '78219', '78220', '78221', '78222', '78223', '78224', '78226', '78227', '78228', '78229', '78230', '78232', '78233', '78237', '78238', '78239', '78240', '78242', '78244', '78245', '78247', '78248', '78249', '78250', '78251', '78252', '78253', '78254', '78256', '78257', '78258', '78259', '78261', '78264', '78266', '78343', '78382', '78404', '78405', '78410', '78411', '78412', '78413', '78414', '78415', '78418', '78501', '78503', '78521', '78526', '78538', '78543', '78550', '78552', '78559', '78566', '78572', '78575', '78576', '78582', '78583', '78584', '78586', '78595', '78602', '78606', '78611', '78612', '78613', '78617', '78620', '78626', '78628', '78633', '78634', '78640', '78641', '78645', '78653', '78654', '78657', '78660', '78664', '78665', '78666', '78669', '78676', '78681', '78702', '78703', '78704', '78705', '78717', '78721', '78723', '78724', '78727', '78728', '78729', '78731', '78732', '78733', '78734', '78735', '78736', '78738', '78741', '78744', '78745', '78746', '78747', '78748', '78749', '78750', '78751', '78752', '78753', '78754', '78756', '78757', '78758', '78759', '78801', '78840', '78942', '79007', '79015', '79072', '79103', '79106', '79107', '79109', '79110', '79201', '79316', '79331', '79382', '79404', '79413', '79416', '79424', '79510', '79549', '79601', '79602', '79603', '79605', '79606', '79714', '79720', '79835', '79836', '79838', '79849', '79902', '79903', '79904', '79905', '79907', '79912', '79915', '79922', '79924', '79925', '79927', '79928', '79930', '79932', '79934', '79935', '79936', '79938', '80002', '80003', '80004', '80005', '80010', '80011', '80012', '80013', '80014', '80015', '80017', '80018', '80020', '80021', '80022', '80023', '80026', '80027', '80030', '80031', '80033', '80102', '80104', '80108', '80109', '80110', '80111', '80112', '80120', '80121', '80123', '80124', '80125', '80126', '80127', '80128', '80129', '80130', '80132', '80133', '80134', '80135', '80138', '80202', '80203', '80204', '80205', '80206', '80207', '80209', '80210', '80211', '80212', '80214', '80215', '80218', '80219', '80220', '80221', '80222', '80223', '80224', '80226', '80227', '80228', '80229', '80231', '80232', '80233', '80234', '80235', '80236', '80237', '80241', '80246', '80247', '80260', '80301', '80302', '80303', '80304', '80305', '80401', '80403', '80421', '80435', '80439', '80452', '80466', '80470', '80501', '80503', '80504', '80513', '80514', '80516', '80521', '80524', '80525', '80526', '80528', '80530', '80534', '80537', '80538', '80542', '80543', '80549', '80550', '80601', '80602', '80615', '80620', '80621', '80631', '80634', '80640', '80651', '80808', '80817', '80829', '80831', '80863', '80903', '80904', '80905', '80906', '80907', '80909', '80910', '80911', '80915', '80916', '80917', '80918', '80919', '80920', '80921', '80922', '80923', '80925', '81001', '81003', '81005', '81008', '81252', '81432', '81501', '81521', '81625', '81635', '81647', '81650', '81652', '82001', '82601', '82604', '82901', '83201', '83202', '83204', '83252', '83263', '83301', '83313', '83318', '83328', '83338', '83341', '83350', '83401', '83402', '83406', '83440', '83442', '83445', '83605', '83607', '83616', '83617', '83634', '83642', '83644', '83646', '83647', '83651', '83655', '83660', '83661', '83672', '83676', '83686', '83687', '83702', '83703', '83704', '83705', '83706', '83709', '83712', '83713', '83714', '83716', '83814', '83815', '83835', '83854', '83861', '83864', '84003', '84004', '84005', '84010', '84014', '84015', '84020', '84025', '84029', '84037', '84040', '84041', '84042', '84043', '84044', '84045', '84047', '84050', '84054', '84057', '84058', '84062', '84065', '84067', '84070', '84074', '84075', '84081', '84084', '84087', '84088', '84094', '84095', '84096', '84097', '84102', '84103', '84104', '84105', '84106', '84107', '84108', '84109', '84111', '84115', '84116', '84117', '84118', '84119', '84120', '84121', '84123', '84124', '84128', '84310', '84401', '84403', '84404', '84405', '84414', '84526', '84601', '84604', '84606', '84651', '84653', '84655', '84660', '84663', '84701', '84720', '84737', '84745', '84770', '84790', '85003', '85006', '85007', '85008', '85009', '85012', '85013', '85014', '85015', '85016', '85017', '85018', '85019', '85020', '85021', '85022', '85023', '85024', '85027', '85028', '85029', '85031', '85032', '85033', '85035', '85037', '85040', '85041', '85042', '85043', '85044', '85048', '85050', '85051', '85053', '85083', '85086', '85087', '85118', '85119', '85120', '85122', '85123', '85128', '85131', '85132', '85138', '85139', '85140', '85142', '85143', '85194', '85201', '85202', '85203', '85204', '85205', '85206', '85207', '85208', '85209', '85210', '85212', '85213', '85215', '85224', '85225', '85226', '85233', '85234', '85248', '85249', '85250', '85251', '85254', '85255', '85257', '85260', '85262', '85266', '85268', '85281', '85282', '85283', '85284', '85286', '85295', '85296', '85301', '85302', '85303', '85304', '85305', '85306', '85307', '85308', '85310', '85323', '85326', '85331', '85335', '85338', '85339', '85340', '85344', '85345', '85349', '85351', '85353', '85355', '85361', '85363', '85373', '85374', '85379', '85381', '85382', '85383', '85388', '85392', '85395', '85396', '85501', '85541', '85546', '85552', '85603', '85614', '85615', '85629', '85635', '85641', '85643', '85650', '85653', '85658', '85701', '85704', '85705', '85706', '85710', '85711', '85712', '85713', '85714', '85715', '85716', '85718', '85719', '85730', '85735', '85737', '85739', '85741', '85742', '85745', '85746', '85747', '85748', '85749', '85750', '85756', '85757', '86001', '86004', '86040', '86046', '86301', '86303', '86305', '86314', '86322', '86323', '86326', '86327', '86335', '86409', '86426', '87002', '87004', '87015', '87031', '87043', '87068', '87102', '87104', '87105', '87106', '87107', '87108', '87109', '87110', '87111', '87112', '87113', '87114', '87120', '87121', '87122', '87123', '87124', '87144', '87401', '87402', '87413', '87505', '87507', '87508', '87544', '87571', '88001', '88005', '88007', '88008', '88011', '88012', '88021', '88044', '88061', '88081', '88101', '88201', '88310', '89002', '89005', '89011', '89012', '89014', '89015', '89027', '89029', '89030', '89031', '89032', '89044', '89052', '89074', '89081', '89084', '89101', '89102', '89103', '89104', '89106', '89107', '89108', '89109', '89110', '89113', '89115', '89117', '89118', '89119', '89120', '89121', '89122', '89123', '89128', '89129', '89130', '89131', '89134', '89135', '89139', '89141', '89142', '89143', '89144', '89145', '89146', '89147', '89148', '89149', '89156', '89169', '89178', '89183', '89403', '89408', '89423', '89431', '89433', '89434', '89436', '89441', '89445', '89460', '89502', '89503', '89506', '89509', '89511', '89512', '89521', '89523', '89701', '89705', '89815', '90001', '90002', '90003', '90004', '90005', '90006', '90007', '90008', '90011', '90012', '90013', '90015', '90016', '90017', '90018', '90019', '90020', '90022', '90023', '90024', '90025', '90026', '90027', '90028', '90029', '90031', '90032', '90033', '90034', '90035', '90036', '90037', '90038', '90039', '90040', '90041', '90042', '90043', '90044', '90045', '90046', '90047', '90048', '90049', '90056', '90057', '90059', '90061', '90062', '90063', '90064', '90065', '90066', '90068', '90069', '90077', '90201', '90210', '90211', '90212', '90220', '90221', '90222', '90230', '90232', '90240', '90241', '90242', '90245', '90247', '90248', '90249', '90250', '90254', '90255', '90260', '90262', '90265', '90266', '90270', '90272', '90275', '90277', '90278', '90280', '90290', '90291', '90292', '90293', '90301', '90302', '90303', '90304', '90305', '90402', '90403', '90404', '90405', '90501', '90502', '90503', '90504', '90505', '90601', '90602', '90603', '90604', '90605', '90606', '90620', '90621', '90623', '90630', '90631', '90638', '90640', '90650', '90660', '90670', '90680', '90701', '90703', '90706', '90710', '90712', '90713', '90715', '90716', '90717', '90720', '90723', '90731', '90732', '90740', '90744', '90745', '90746', '90755', '90802', '90804', '90805', '90806', '90807', '90808', '90810', '90813', '90814', '90815', '91001', '91006', '91007', '91010', '91011', '91016', '91020', '91024', '91030', '91040', '91042', '91101', '91103', '91104', '91105', '91106', '91107', '91201', '91202', '91203', '91204', '91205', '91206', '91207', '91208', '91214', '91301', '91302', '91304', '91306', '91307', '91311', '91316', '91320', '91321', '91324', '91325', '91326', '91331', '91335', '91340', '91342', '91343', '91344', '91345', '91350', '91351', '91352', '91354', '91355', '91356', '91360', '91361', '91362', '91364', '91367', '91377', '91381', '91384', '91387', '91390', '91401', '91402', '91403', '91405', '91406', '91411', '91423', '91436', '91501', '91502', '91504', '91505', '91506', '91601', '91602', '91604', '91606', '91607', '91701', '91702', '91706', '91709', '91710', '91711', '91722', '91723', '91724', '91730', '91731', '91732', '91733', '91737', '91739', '91740', '91741', '91744', '91745', '91746', '91748', '91750', '91752', '91754', '91755', '91761', '91762', '91763', '91764', '91765', '91766', '91767', '91768', '91770', '91773', '91775', '91776', '91780', '91784', '91786', '91789', '91791', '91792', '91801', '91803', '91901', '91902', '91910', '91911', '91913', '91914', '91932', '91941', '91942', '91945', '91950', '91977', '92003', '92007', '92008', '92009', '92010', '92011', '92014', '92019', '92020', '92021', '92024', '92025', '92026', '92027', '92028', '92029', '92037', '92040', '92054', '92056', '92057', '92058', '92064', '92065', '92069', '92071', '92075', '92078', '92081', '92082', '92083', '92084', '92101', '92102', '92103', '92104', '92105', '92106', '92107', '92108', '92109', '92110', '92111', '92113', '92114', '92115', '92116', '92117', '92118', '92119', '92120', '92121', '92122', '92123', '92124', '92126', '92127', '92128', '92129', '92130', '92131', '92139', '92154', '92173', '92201', '92203', '92211', '92220', '92223', '92225', '92234', '92236', '92240', '92241', '92252', '92254', '92256', '92260', '92262', '92264', '92277', '92284', '92301', '92307', '92308', '92311', '92313', '92316', '92320', '92324', '92325', '92335', '92336', '92337', '92342', '92344', '92345', '92346', '92352', '92354', '92359', '92363', '92371', '92373', '92374', '92376', '92377', '92382', '92392', '92394', '92395', '92399', '92404', '92405', '92407', '92408', '92410', '92411', '92501', '92503', '92504', '92505', '92506', '92507', '92508', '92509', '92530', '92532', '92543', '92544', '92545', '92551', '92553', '92555', '92557', '92562', '92563', '92567', '92570', '92571', '92582', '92583', '92584', '92585', '92586', '92587', '92591', '92592', '92595', '92596', '92602', '92603', '92604', '92606', '92610', '92612', '92614', '92618', '92620', '92624', '92625', '92626', '92627', '92629', '92630', '92637', '92646', '92647', '92648', '92649', '92651', '92653', '92655', '92656', '92660', '92663', '92672', '92673', '92675', '92677', '92679', '92683', '92688', '92691', '92692', '92701', '92703', '92704', '92705', '92706', '92707', '92708', '92780', '92782', '92801', '92802', '92804', '92805', '92806', '92807', '92821', '92823', '92831', '92832', '92833', '92835', '92840', '92841', '92843', '92844', '92845', '92860', '92865', '92866', '92867', '92868', '92869', '92870', '92879', '92880', '92881', '92882', '92883', '92886', '92887', '93001', '93003', '93004', '93010', '93012', '93015', '93021', '93022', '93023', '93030', '93033', '93035', '93036', '93041', '93060', '93063', '93065', '93203', '93204', '93212', '93215', '93219', '93221', '93223', '93230', '93241', '93245', '93247', '93250', '93257', '93268', '93274', '93277', '93280', '93286', '93291', '93292', '93304', '93305', '93306', '93307', '93308', '93309', '93311', '93313', '93314', '93422', '93436', '93446', '93451', '93454', '93505', '93514', '93534', '93535', '93536', '93543', '93550', '93551', '93552', '93555', '93560', '93561', '93602', '93610', '93611', '93612', '93618', '93619', '93620', '93622', '93630', '93631', '93635', '93636', '93637', '93638', '93640', '93646', '93648', '93654', '93656', '93657', '93662', '93701', '93702', '93703', '93704', '93705', '93706', '93710', '93711', '93720', '93722', '93725', '93726', '93727', '93728', '93730', '93901', '93905', '93906', '93907', '93908', '93923', '93924', '93926', '93927', '93930', '93933', '93940', '93950', '93955', '93960', '94002', '94005', '94010', '94015', '94019', '94022', '94025', '94030', '94040', '94043', '94044', '94061', '94062', '94063', '94066', '94070', '94080', '94085', '94086', '94087', '94089', '94102', '94103', '94107', '94108', '94109', '94110', '94112', '94114', '94115', '94116', '94117', '94118', '94121', '94122', '94123', '94127', '94131', '94132', '94133', '94134', '94303', '94306', '94401', '94402', '94403', '94404', '94501', '94502', '94503', '94505', '94506', '94507', '94509', '94510', '94513', '94517', '94518', '94519', '94520', '94521', '94523', '94525', '94526', '94530', '94531', '94533', '94534', '94536', '94538', '94541', '94542', '94544', '94545', '94546', '94547', '94550', '94551', '94552', '94553', '94555', '94556', '94558', '94559', '94560', '94561', '94564', '94565', '94566', '94568', '94572', '94577', '94578', '94579', '94580', '94582', '94583', '94585', '94587', '94588', '94589', '94590', '94591', '94595', '94596', '94597', '94598', '94601', '94602', '94603', '94605', '94606', '94607', '94608', '94619', '94621', '94801', '94803', '94804', '94805', '94806', '94901', '94903', '94904', '94920', '94925', '94928', '94930', '94931', '94939', '94941', '94945', '94947', '94949', '94952', '94954', '94960', '94965', '95003', '95008', '95010', '95014', '95019', '95020', '95023', '95030', '95032', '95035', '95037', '95050', '95051', '95054', '95060', '95062', '95070', '95076', '95110', '95111', '95112', '95116', '95117', '95118', '95119', '95120', '95121', '95122', '95123', '95124', '95125', '95126', '95127', '95128', '95129', '95130', '95131', '95132', '95133', '95134', '95135', '95136', '95138', '95148', '95203', '95204', '95205', '95206', '95207', '95209', '95210', '95215', '95219', '95220', '95240', '95242', '95258', '95301', '95304', '95307', '95315', '95316', '95320', '95322', '95326', '95330', '95334', '95336', '95337', '95338', '95340', '95341', '95348', '95350', '95351', '95354', '95355', '95356', '95357', '95358', '95360', '95361', '95363', '95366', '95367', '95368', '95370', '95376', '95377', '95380', '95382', '95386', '95388', '95391', '95401', '95403', '95404', '95405', '95407', '95409', '95425', '95436', '95446', '95448', '95451', '95467', '95472', '95476', '95492', '95503', '95602', '95603', '95605', '95608', '95610', '95616', '95618', '95619', '95620', '95621', '95623', '95624', '95626', '95628', '95630', '95632', '95648', '95650', '95658', '95660', '95661', '95662', '95667', '95670', '95672', '95673', '95677', '95678', '95682', '95687', '95688', '95691', '95692', '95693', '95694', '95695', '95709', '95713', '95722', '95726', '95746', '95747', '95757', '95758', '95762', '95765', '95776', '95811', '95815', '95816', '95817', '95818', '95819', '95820', '95821', '95822', '95823', '95824', '95825', '95826', '95827', '95828', '95829', '95831', '95832', '95833', '95834', '95835', '95838', '95841', '95842', '95843', '95864', '95901', '95928', '95932', '95945', '95946', '95948', '95949', '95953', '95959', '95963', '95965', '95987', '95988', '95993', '96001', '96002', '96003', '96007', '96019', '96021', '96022', '96080', '96150', '96161', '96701', '96706', '96707', '96712', '96717', '96731', '96734', '96744', '96782', '96786', '96789', '96791', '96792', '96795', '96797', '96813', '96814', '96815', '96816', '96817', '96818', '96821', '96822', '96825', '96826', '97004', '97005', '97006', '97007', '97008', '97015', '97027', '97030', '97031', '97035', '97045', '97056', '97058', '97060', '97062', '97068', '97070', '97071', '97080', '97086', '97089', '97115', '97116', '97123', '97124', '97127', '97128', '97132', '97138', '97140', '97141', '97146', '97201', '97202', '97203', '97205', '97206', '97209', '97210', '97211', '97212', '97213', '97214', '97215', '97216', '97217', '97218', '97219', '97220', '97221', '97222', '97223', '97224', '97225', '97229', '97230', '97231', '97232', '97233', '97236', '97239', '97266', '97267', '97301', '97302', '97303', '97305', '97306', '97321', '97322', '97325', '97330', '97333', '97352', '97355', '97365', '97370', '97378', '97381', '97386', '97401', '97402', '97404', '97405', '97408', '97424', '97426', '97448', '97477', '97478', '97487', '97503', '97526', '97527', '97701', '97702', '97739', '97756', '98001', '98002', '98003', '98004', '98005', '98006', '98007', '98008', '98010', '98011', '98012', '98019', '98021', '98022', '98023', '98026', '98027', '98028', '98029', '98030', '98031', '98032', '98033', '98034', '98036', '98037', '98038', '98040', '98042', '98043', '98045', '98047', '98052', '98053', '98055', '98056', '98057', '98058', '98059', '98065', '98072', '98074', '98075', '98077', '98087', '98092', '98101', '98102', '98103', '98104', '98105', '98106', '98107', '98108', '98109', '98112', '98115', '98116', '98117', '98118', '98119', '98121', '98122', '98125', '98133', '98136', '98144', '98146', '98148', '98155', '98166', '98168', '98177', '98178', '98188', '98198', '98199', '98201', '98203', '98204', '98208', '98221', '98223', '98225', '98226', '98229', '98230', '98232', '98233', '98239', '98245', '98247', '98248', '98250', '98252', '98257', '98258', '98261', '98264', '98270', '98271', '98272', '98273', '98274', '98275', '98277', '98281', '98282', '98284', '98290', '98292', '98296', '98310', '98311', '98312', '98321', '98327', '98328', '98329', '98332', '98333', '98335', '98337', '98338', '98354', '98359', '98360', '98365', '98366', '98367', '98368', '98370', '98371', '98372', '98373', '98374', '98375', '98383', '98387', '98388', '98390', '98391', '98392', '98402', '98403', '98404', '98405', '98406', '98407', '98408', '98409', '98418', '98422', '98424', '98443', '98444', '98445', '98446', '98465', '98466', '98467', '98498', '98499', '98501', '98502', '98503', '98506', '98512', '98513', '98516', '98520', '98528', '98531', '98532', '98550', '98576', '98577', '98579', '98580', '98584', '98589', '98596', '98597', '98604', '98607', '98611', '98626', '98629', '98642', '98648', '98660', '98661', '98662', '98663', '98664', '98665', '98671', '98672', '98675', '98682', '98683', '98684', '98685', '98686', '98802', '98837', '98841', '98844', '98855', '98902', '98903', '98908', '98922', '98937', '98942', '99001', '99004', '99005', '99006', '99019', '99021', '99022', '99025', '99026', '99027', '99037', '99156', '99201', '99202', '99203', '99204', '99205', '99206', '99207', '99208', '99212', '99216', '99217', '99223', '99224', '99320', '99336', '99344', '99352', '99354', '99362', '99501', '99502', '99504', '99507', '99508', '99515', '99516', '99517', '99518', '99567', '99577', '99645', '99654', '99701', '99705']

In [2187]:
n

['01001',
 '01005',
 '01007',
 '01013',
 '01020',
 '01028',
 '01030',
 '01040',
 '01056',
 '01057',
 '01060',
 '01075',
 '01077',
 '01085',
 '01089',
 '01104',
 '01105',
 '01107',
 '01108',
 '01109',
 '01118',
 '01119',
 '01129',
 '01151',
 '01331',
 '01420',
 '01430',
 '01432',
 '01440',
 '01450',
 '01452',
 '01453',
 '01460',
 '01462',
 '01463',
 '01464',
 '01469',
 '01473',
 '01475',
 '01501',
 '01504',
 '01505',
 '01506',
 '01507',
 '01510',
 '01516',
 '01518',
 '01519',
 '01520',
 '01522',
 '01523',
 '01524',
 '01527',
 '01529',
 '01532',
 '01534',
 '01535',
 '01540',
 '01541',
 '01550',
 '01560',
 '01562',
 '01564',
 '01569',
 '01571',
 '01585',
 '01588',
 '01590',
 '01602',
 '01603',
 '01604',
 '01605',
 '01606',
 '01607',
 '01609',
 '01610',
 '01702',
 '01720',
 '01730',
 '01740',
 '01741',
 '01746',
 '01747',
 '01748',
 '01754',
 '01756',
 '01757',
 '01770',
 '01773',
 '01775',
 '01776',
 '01778',
 '01801',
 '01803',
 '01810',
 '01821',
 '01824',
 '01826',
 '01830',
 '01832',


### Prediction Accuracy

In [1820]:
top_pred = pd.Series(np.exp(top_model_ols.predict(top_x_test)), index = top_y_test.index)
print("Average Prediction Accuracy: %.2f percent" %((1 - np.abs(np.exp(top_y_test)/top_pred-1)).mean() * 100))

Average Prediction Accuracy: 97.64 percent


## Try deleting older data (2011 and 2012)

In [1169]:
new_train = total[total['year'] == '2015'].drop(['zip','year'], axis = 1)
new_test = total[total['year'] == '2016'].drop(['zip','year'], axis = 1)

In [1170]:
new_x_train = new_train.iloc[:,:-1]
new_x_test = new_test.iloc[:,:-1]

new_y_train = new_train.iloc[:,-1]
new_y_test = new_test.iloc[:,-1]

### Log Everything

In [1171]:
new_x_train = np.log(new_x_train+1)
new_x_test = np.log(new_x_test+1)
new_y_train = np.log(new_y_train)
new_y_test = np.log(new_y_test)

In [1172]:
new_model_ols = LinearRegression()
new_model_lasso = Lasso()
new_model_ridge = Ridge()
new_model_gradient = GradientBoostingRegressor()

new_model_ols.fit(new_x_train, new_y_train)
new_model_lasso.fit(new_x_train, new_y_train)
new_model_ridge.fit(new_x_train, new_y_train)
new_model_gradient.fit(new_x_train, new_y_train)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
                          learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='auto',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

### Train Score

In [1173]:
print('Simple Linear Train Score: %.4f'%new_model_ols.score(new_x_train, new_y_train))
print('Lasso Train Score: %.4f'%new_model_lasso.score(new_x_train, new_y_train))
print('Ridge Train Score: %.4f'%new_model_ridge.score(new_x_train, new_y_train))
print('Gradient Regressor Train Score: %.4f'%new_model_gradient.score(new_x_train, new_y_train))

Simple Linear Train Score: 0.9951
Lasso Train Score: 0.0000
Ridge Train Score: 0.9949
Gradient Regressor Train Score: 0.9953


### Test Score

In [1174]:
print('Simple Linear Test Score: %.4f'%new_model_ols.score(new_x_test, new_y_test))
print('Lasso Test Score: %.4f'%new_model_lasso.score(new_x_train, new_y_train))
print('Ridge Test Score: %.4f'%new_model_ridge.score(new_x_train, new_y_train))
print('Gradient Regressor Test Score: %.4f'%new_model_gradient.score(new_x_train, new_y_train))

Simple Linear Test Score: 0.9917
Lasso Test Score: 0.0000
Ridge Test Score: 0.9949
Gradient Regressor Test Score: 0.9953


### RMSE

In [2049]:
print('Simple Linear RMSE Score: %.5f' % sqrt(mean_squared_error(np.exp(new_model_ols.predict(new_x_test)), np.exp(new_y_test))))
print('Lasso RMSE Score: %.5f' % sqrt(mean_squared_error(np.exp(new_model_lasso.predict(new_x_test)), np.exp(new_y_test))))
print('Ridge RMSE Score: %.5f' % sqrt(mean_squared_error(np.exp(new_model_ridge.predict(new_x_test)), np.exp(new_y_test))))
print('Gradient Regressor RMSE Score: %.5f' % sqrt(mean_squared_error(np.exp(new_model_gradient.predict(new_x_test)), np.exp(new_y_test))))



Simple Linear RMSE Score: 50.52294
Lasso RMSE Score: 610.54952
Ridge RMSE Score: 50.60957
Gradient Regressor RMSE Score: 49.79466


### Prediction Accuracy

In [1176]:
new_pred = pd.Series(np.exp(new_model_gradient.predict(new_x_test)), index = new_y_test.index)
print("Average Prediction Accuracy: %.2f percent" %((1 - np.abs(np.exp(new_y_test)/new_pred-1)).mean() * 100))

Average Prediction Accuracy: 97.60 percent


# IRS Data

In [1821]:
irs2011 = pd.read_csv('11zpallagi.csv')
irs2012 = pd.read_csv('12zpallagi.csv')
irs2013 = pd.read_csv('13zpallagi.csv')
irs2014 = pd.read_csv('14zpallagi.csv')
irs2015 = pd.read_csv('15zpallagi.csv')
irs2016 = pd.read_csv('16zpallagi.csv')
irs2017 = pd.read_csv('17zpallagi.csv')

irs_mutual_columns = set(irs2013.columns).\
intersection(set(irs2014.columns)).intersection(set(irs2015.columns)).intersection(set(irs2016.columns)).\
intersection(set(irs2017.columns))

irs2013 = irs2013[irs_mutual_columns].drop(['STATEFIPS','STATE'], axis = 1)
irs2014 = irs2014[irs_mutual_columns].drop(['STATEFIPS','STATE'], axis = 1)
irs2015 = irs2015[irs_mutual_columns].drop(['STATEFIPS','STATE'], axis = 1)
irs2016 = irs2016[irs_mutual_columns].drop(['STATEFIPS','STATE'], axis = 1)
irs2017 = irs2017[irs_mutual_columns].drop(['STATEFIPS','STATE'], axis = 1)

irs2013.columns = irs2013.columns.str.replace('zipcode','zip')
irs2014.columns = irs2014.columns.str.replace('zipcode','zip')
irs2015.columns = irs2015.columns.str.replace('zipcode','zip')
irs2016.columns = irs2016.columns.str.replace('zipcode','zip')
irs2017.columns = irs2017.columns.str.replace('zipcode','zip')

irs2013 = irs2013.groupby('zip').sum()
irs2014 = irs2014.groupby('zip').sum()
irs2015 = irs2015.groupby('zip').sum()
irs2016 = irs2016.groupby('zip').sum()
irs2017 = irs2017.groupby('zip').sum()


In [1822]:
irs2013.columns = [i + '_2013' for i in irs2013.columns]
irs2014.columns = [i + '_2014' for i in irs2014.columns]
irs2015.columns = [i + '_2015' for i in irs2015.columns]
irs2016.columns = [i + '_2016' for i in irs2016.columns]
irs2017.columns = [i + '_2017' for i in irs2017.columns]

In [1823]:
taxes2015 = pd.concat([irs2015,irs2014,irs2013], join = 'inner', axis = 1).reset_index()
taxes2016 = pd.concat([irs2016,irs2015,irs2014], join = 'inner', axis = 1).reset_index()
taxes2017 = pd.concat([irs2017,irs2016,irs2015], join = 'inner', axis = 1).reset_index()

In [1824]:
taxes2015['year'] = '2015'
taxes2016['year'] = '2016'
taxes2017['year'] = '2017'

In [1825]:
taxes2015.columns = taxes2015.columns.str.replace('_2015','_3y').str.replace('_2014','_4y').str.replace('_2013','_5y')
taxes2016.columns = taxes2016.columns.str.replace('_2016','_3y').str.replace('_2015','_4y').str.replace('_2014','_5y')
taxes2017.columns = taxes2017.columns.str.replace('_2017','_3y').str.replace('_2016','_4y').str.replace('_2015','_5y')



In [1826]:
irs_train = zri_train_2015.merge(taxes2015, on  = ['zip','year']).merge(zillow_mean, how = 'inner', on = ['zip','year'])


In [1827]:
irs_test = zri_train_2016.merge(taxes2016, on  = ['zip','year']).merge(zillow_mean, how = 'inner', on = ['zip','year'])



In [1828]:
irs_train['zip'] = [str(i) for i in irs_train.zip]
irs_train['zip_year'] = irs_train.zip + '_' + irs_train.year
irs_train = irs_train.drop(['zip','year'], axis = 1).set_index('zip_year')

In [1829]:
irs_test['zip'] = [str(i) for i in irs_test.zip]
irs_test['zip_year'] = irs_test.zip + '_' + irs_test.year
irs_test = irs_test.drop(['zip','year'], axis = 1).set_index('zip_year')

In [1830]:
irs_train = irs_train[(irs_train < 0).sum(axis = 1) == 0]
irs_test = irs_test[(irs_test < 0).sum(axis = 1) == 0]

## Models Building (IRS)

In [1831]:
irs_x_train = irs_train.iloc[:,:-1]
irs_y_train = irs_train.iloc[:,-1]

In [1832]:
irs_x_test = irs_test.iloc[:,:-1]
irs_y_test = irs_test.iloc[:,-1]

### Log Evertyhing

In [1833]:
irs_x_train = np.log(irs_x_train+1)
irs_x_test = np.log(irs_x_test+1)

irs_y_train = np.log(irs_y_train)
irs_y_test = np.log(irs_y_test)

In [1834]:
irs_model_ols = LinearRegression()
irs_model_lasso = Lasso()
irs_model_ridge = Ridge()
irs_model_gradient = GradientBoostingRegressor()

irs_model_ols.fit(irs_x_train, irs_y_train)
irs_model_lasso.fit(irs_x_train, irs_y_train)
irs_model_ridge.fit(irs_x_train, irs_y_train)
irs_model_gradient.fit(irs_x_train, irs_y_train)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
                          learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='auto',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

### Train Score

In [1320]:
print('Simple Linear Train Score: %.4f'%irs_model_ols.score(irs_x_train, irs_y_train))
print('Lasso Train Score: %.4f'%irs_model_lasso.score(irs_x_train, irs_y_train))
print('Ridge Train Score: %.4f'%irs_model_ridge.score(irs_x_train, irs_y_train))
print('Gradient Regressor Train Score: %.4f'%irs_model_gradient.score(irs_x_train, irs_y_train))

Simple Linear Train Score: 0.9949
Lasso Train Score: 0.0000
Ridge Train Score: 0.9947
Gradient Regressor Train Score: 0.9949


### Test Score

In [1321]:
print('Simple Linear Test Score: %.4f'%irs_model_ols.score(irs_x_test, irs_y_test))
print('Lasso Test Score: %.4f'%irs_model_lasso.score(irs_x_test, irs_y_test))
print('Ridge Test Score: %.4f'%irs_model_ridge.score(irs_x_test, irs_y_test))
print('Gradient Regressor Test Score: %.4f'%irs_model_gradient.score(irs_x_test, irs_y_test))

Simple Linear Test Score: 0.9919
Lasso Test Score: -0.0009
Ridge Test Score: 0.9917
Gradient Regressor Test Score: 0.9917


### RMSE

In [1911]:
print('Simple Linear RMSE Score: %.5f' % sqrt(mean_squared_error(np.exp(irs_model_ols.predict(irs_x_test)), np.exp(irs_y_test))))
print('Lasso RMSE Score: %.5f' % sqrt(mean_squared_error(np.exp(irs_model_lasso.predict(irs_x_test)), np.exp(irs_y_test))))
print('Ridge RMSE Score: %.5f' % sqrt(mean_squared_error(np.exp(irs_model_ridge.predict(irs_x_test)), np.exp(irs_y_test))))
print('Gradient Regressor RMSE Score: %.5f' % sqrt(mean_squared_error(np.exp(irs_model_gradient.predict(irs_x_test)), np.exp(irs_y_test))))



Simple Linear RMSE Score: 51.39572
Lasso RMSE Score: 611.32553
Ridge RMSE Score: 52.43858
Gradient Regressor RMSE Score: 51.16028


### Accuracy

In [1842]:
irs_pred = pd.Series(np.exp(irs_model_ols.predict(irs_x_test)), index = irs_y_test.index)
print("Average Prediction Accuracy: %.2f percent" %((1 - np.abs(np.exp(irs_y_test)/irs_pred-1)).mean() * 100))

Average Prediction Accuracy: 97.57 percent


## Top Features From IRS

In [1955]:
sorted(list(zip(irs_x_train.columns, irs_model_gradient.feature_importances_)), key = lambda x: x[1], reverse = True)[0:10]



[('zri_1y', 0.997450481974797),
 ('zri_3y', 0.0009439677465665153),
 ('zri_2y', 0.0006102681556737682),
 ('A07260_3y', 8.886497565459614e-05),
 ('SCHF_4y', 7.253273690399848e-05),
 ('A02300_5y', 6.546155420672393e-05),
 ('A19300_5y', 5.2950426072523325e-05),
 ('A03210_4y', 4.4430101607438114e-05),
 ('SCHF_3y', 4.2061153776998855e-05),
 ('N18450_5y', 3.695146715501609e-05)]

In [1956]:
top_irs = sorted(list(zip(irs_x_train.columns, irs_model_gradient.feature_importances_)), key = lambda x: x[1], reverse = True)[0:20]



In [1957]:
top_irs_x_train = irs_x_train[[i[0] for i in top_irs]]
top_irs_x_test = irs_x_test[[i[0] for i in top_irs]]

In [1958]:
top_irs_y_train = irs_y_train.copy()
top_irs_y_test = irs_y_test.copy()

In [1959]:
top_irs_model_ols = LinearRegression()
top_irs_model_lasso = Lasso()
top_irs_model_ridge = Ridge()
top_irs_model_gradient = GradientBoostingRegressor()

top_irs_model_ols.fit(top_irs_x_train, top_irs_y_train)
top_irs_model_lasso.fit(top_irs_x_train, top_irs_y_train)
top_irs_model_ridge.fit(top_irs_x_train, top_irs_y_train)
top_irs_model_gradient.fit(top_irs_x_train, top_irs_y_train)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
                          learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='auto',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

### Train Score

In [1960]:
print('Simple Linear Train Score: %.4f'%top_irs_model_ols.score(top_irs_x_train, top_irs_y_train))
print('Lasso Train Score: %.4f'%top_irs_model_lasso.score(top_irs_x_train, top_irs_y_train))
print('Ridge Train Score: %.4f'%top_irs_model_ridge.score(top_irs_x_train, top_irs_y_train))
print('Gradient Regressor Train Score: %.4f'%top_irs_model_gradient.score(top_irs_x_train, top_irs_y_train))

Simple Linear Train Score: 0.9942
Lasso Train Score: 0.0000
Ridge Train Score: 0.9941
Gradient Regressor Train Score: 0.9948


### Test Score

In [1961]:
print('Simple Linear Test Score: %.4f'%top_irs_model_ols.score(top_irs_x_test, top_irs_y_test))
print('Lasso Test Score: %.4f'%top_irs_model_lasso.score(top_irs_x_test, top_irs_y_test))
print('Ridge Test Score: %.4f'%top_irs_model_ridge.score(top_irs_x_test, top_irs_y_test))
print('Gradient Regressor Test Score: %.4f'%top_irs_model_gradient.score(top_irs_x_test, top_irs_y_test))

Simple Linear Train Score: 0.9921
Lasso Train Score: -0.0009
Ridge Train Score: 0.9919
Gradient Regressor Train Score: 0.9918


### RMSE

In [1962]:
print('Simple Linear RMSE Score: %.5f' % sqrt(mean_squared_error(np.exp(top_irs_model_ols.predict(top_irs_x_test)), np.exp(top_irs_y_test))))
print('Lasso RMSE Score: %.5f' % sqrt(mean_squared_error(np.exp(top_irs_model_lasso.predict(top_irs_x_test)), np.exp(top_irs_y_test))))
print('Ridge RMSE Score: %.5f' % sqrt(mean_squared_error(np.exp(top_irs_model_ridge.predict(top_irs_x_test)), np.exp(top_irs_y_test))))
print('Gradient Regressor RMSE Score: %.5f' % sqrt(mean_squared_error(np.exp(top_irs_model_gradient.predict(top_irs_x_test)), np.exp(top_irs_y_test))))


Simple Linear Kaggle Score: 50.34790
Lasso Kaggle Score: 611.32553
Ridge Kaggle Score: 51.17912
Gradient Regressor Kaggle Score: 51.14325


### Prediction Accuracy

In [1963]:
top_irs_pred = pd.Series(np.exp(top_irs_model_ols.predict(top_irs_x_test)), index = top_irs_y_test.index)
print("Average Prediction Accuracy: %.2f percent" %((1 - np.abs(np.exp(top_irs_y_test)/top_irs_pred-1)).mean() * 100))

Average Prediction Accuracy: 97.60 percent


## Take Average of Census and IRS Predictions

### Full Census and Full IRS

In [1689]:
together = pd.concat([pred, irs_pred],join = 'inner', axis = 1)
together.columns = ['census','irs']

In [1855]:
together['average'] = (together.census + together.irs)/2

### RMSE

In [1864]:
together = pd.concat([together.average, np.exp(y_test)], join = 'inner', axis = 1)

In [2234]:
print("Average Root Mean Squared Error: %.3f" %sqrt(mean_squared_error(together['ZRI+3'],together.average)))

Average Root Mean Squared Error: 49.178


In [2233]:
from sklearn.metrics import r2_score

r2_score(together['ZRI+3'],together.average)


0.9933098284639811

In [1886]:
print("Average Prediction Accuracy: %.2f percent" %((1 - np.abs(np.exp(y_test)/together.average - 1)).mean()*100))

Average Prediction Accuracy: 97.67 percent


### Top 10 Census and Top 10 IRS

In [1887]:
top_together = pd.concat([top_pred, top_irs_pred],join = 'inner', axis = 1)
top_together.columns = ['census','irs']

In [1888]:
top_together['average'] = (top_together.census + top_together.irs)/2

In [1982]:
top_together = pd.concat([top_together.average, np.exp(y_test)], join = 'inner', axis = 1)

In [2236]:
print("Average Root Mean Squared Error is: %.3f" %sqrt(mean_squared_error(top_together['ZRI+3'],top_together.average,)))




Average Root Mean Squared Error is: 47.201


In [2239]:
r2_score(top_together['ZRI+3'],top_together.average)

0.9938368648297565

In [1891]:
print("Average Prediction Accuracy: %.2f percent" %((1 - np.abs(np.exp(y_test)/top_together.average - 1)).mean()*100))

Average Prediction Accuracy: 97.70 percent


## Full Model (top 10 irs + top 10 census)

In [2016]:
top_census_features = [i[0] for i in top_features]
top_census_features

['zri_1y',
 'owner_occupied_housing_units_median_value_5y',
 'owner_occupied_housing_units_upper_value_quartile_5y',
 'median_rent_5y',
 'median_rent_4y',
 'housing_built_2005_or_later_5y',
 'median_rent_3y',
 'housing_built_2005_or_later_4y',
 'income_per_capita_3y',
 'percent_income_spent_on_rent_3y']

In [2017]:
top_irs_features = [i[0] for i in top_irs][0:10]
top_irs_features

['zri_1y',
 'zri_3y',
 'zri_2y',
 'A07260_3y',
 'SCHF_4y',
 'A02300_5y',
 'A19300_5y',
 'A03210_4y',
 'SCHF_3y',
 'N18450_5y']

In [1965]:
full_top_features = list(set(top_census_features).union(set(top_irs_features)))

In [1966]:
total

,zip,zri_3y,zri_2y,zri_1y,year,pop_in_labor_force_3y,population_3_years_over_3y,female_65_to_66_3y,rent_10_to_15_percent_3y,family_households_3y,...,male_18_to_19_5y,occupation_sales_office_5y,two_or_more_races_pop_5y,owner_occupied_housing_units_lower_value_quartile_5y,income_per_capita_5y,female_22_to_24_5y,rent_over_50_percent_5y,poverty_5y,households_retirement_income_5y,ZRI+3
5302,1001,1196.583333,1226.583333,1333.583333,2013,9607,16771,139,214,4085,...,195,2478,132,174200.0,28931.0,265,405,1428,1319,1380.166667
15153,1001,1226.583333,1333.583333,1380.166667,2014,9556,16635,161,198,4168,...,264,2499,146,169400.0,29158.0,307,382,1251,1347,1421.833333
25004,1001,1333.583333,1380.166667,1421.833333,2015,9600,16962,143,219,4080,...,161,2511,119,166400.0,28457.0,431,550,1537,1285,1338.500000
34855,1001,1380.166667,1421.833333,1338.500000,2016,9211,17029,260,169,3943,...,159,2367,133,167500.0,29444.0,415,600,1390,1240,1362.416667
38744,1005,1268.833333,1282.833333,1282.833333,2016,2988,5156,94,16,1150,...,104,625,1,161700.0,29004.0,43,65,224,280,1358.833333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35007,99701,1504.416667,1482.083333,1477.250000,2016,10452,17831,112,222,3807,...,214,2040,1680,132400.0,29907.0,416,956,2323,1589,1506.416667
4920,99705,1411.916667,1348.000000,1479.750000,2013,13267,22794,157,90,5780,...,363,2442,1225,151400.0,30177.0,449,244,1226,1640,1502.666667
14771,99705,1348.000000,1479.750000,1502.666667,2014,13457,22948,178,143,5747,...,382,2676,1473,145500.0,30358.0,495,356,1180,1554,1474.750000
24622,99705,1479.750000,1502.666667,1474.750000,2015,13745,23095,87,125,5740,...,356,2473,1585,151000.0,30768.0,437,352,1328,1788,1479.583333


In [1967]:
### Join Data Sets

In [1968]:
census_x_train = total[total['year'] == '2015']
census_x_train['zip'] = [str(i) for i in census_x_train.zip]
census_x_train['zip_year'] = census_x_train.zip + '_' + census_x_train.year
census_x_train = census_x_train.drop(['zip','year'], axis = 1).set_index('zip_year')

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [1969]:
full_train = pd.concat([census_x_train.drop(['zri_1y','zri_2y','zri_3y','ZRI+3'], axis = 1),irs_train], join = 'inner', axis = 1)[full_top_features+['ZRI+3']]


In [1970]:
full_train.head()

,A03210_4y,income_per_capita_3y,median_rent_4y,N18450_5y,zri_2y,zri_3y,housing_built_2005_or_later_4y,A02300_5y,housing_built_2005_or_later_5y,SCHF_3y,SCHF_4y,zri_1y,A19300_5y,owner_occupied_housing_units_upper_value_quartile_5y,owner_occupied_housing_units_median_value_5y,median_rent_5y,percent_income_spent_on_rent_3y,median_rent_3y,A07260_3y,ZRI+3
zip_year,,,,,,,,,,,,,,,,,,,,
1001_2015,958.0,30044.0,800.0,190.0,1380.166667,1333.583333,59,5500.0,63,0.0,0.0,1421.833333,16302.0,265700.0,213000.0,802.0,31.5,809.0,126.0,1338.500000
1007_2015,940.0,34278.0,755.0,110.0,1527.250000,1599.375000,43,3936.0,15,50.0,50.0,1490.125000,20527.0,321500.0,260000.0,743.0,33.4,778.0,507.0,1504.500000
1013_2015,921.0,21477.0,730.0,100.0,1159.083333,1108.416667,84,7435.0,10,0.0,0.0,1226.416667,9581.0,208900.0,179800.0,689.0,30.9,731.0,43.0,1208.916667
1020_2015,1399.0,28032.0,710.0,190.0,1177.833333,1128.833333,16,9981.0,0,0.0,0.0,1243.750000,19001.0,230300.0,181400.0,699.0,28.9,718.0,62.0,1205.250000
1028_2015,971.0,38300.0,678.0,190.0,1554.500000,1567.833333,55,4260.0,39,0.0,0.0,1565.916667,23286.0,351100.0,256300.0,826.0,33.1,675.0,122.0,1483.666667


In [1971]:
census_x_test = total[total['year'] == '2016']
census_x_test['zip'] = [str(i) for i in census_x_test.zip]
census_x_test['zip_year'] = census_x_test.zip + '_' + census_x_test.year
census_x_test = census_x_test.drop(['zip','year'], axis = 1).set_index('zip_year')

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [1972]:
full_test = pd.concat([census_x_test.drop(['zri_1y','zri_2y','zri_3y','ZRI+3'], axis = 1), irs_test], join = 'inner', axis = 1)[full_top_features+['ZRI+3']]


In [1973]:
full_x_train = full_train.iloc[:,:-1]
full_x_test = full_test.iloc[:,:-1]

full_y_train = full_train.iloc[:,-1]
full_y_test = full_test.iloc[:,-1]

#### Log Everything

In [1974]:
full_x_train = np.log(full_x_train+1)
full_x_test = np.log(full_x_test+1)

full_y_train = np.log(full_y_train)
full_y_test = np.log(full_y_test)

In [1975]:
full_model_ols = LinearRegression()
full_model_ridge = Ridge()
full_model_gradient = GradientBoostingRegressor()

full_model_ols.fit(full_x_train, full_y_train)
full_model_ridge.fit(full_x_train, full_y_train)
full_model_gradient.fit(full_x_train, full_y_train)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
                          learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='auto',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

### R^2

In [1976]:
print('Simple Linear Test Score: %.4f'%full_model_ols.score(full_x_test, full_y_test))
print('Ridge Test Score: %.4f'%full_model_ridge.score(full_x_test, full_y_test))
print('Gradient Regressor Test Score: %.4f'%full_model_gradient.score(full_x_test, full_y_test))

Simple Linear Test Score: 0.9923
Ridge Test Score: 0.9921
Gradient Regressor Test Score: 0.9921


### RMSE

In [2044]:
print('Simple Linear RMSE Score: %.5f' % sqrt(mean_squared_error(np.exp(full_model_ols.predict(full_x_test)), np.exp(full_y_test))))
print('Ridge RMSE Score: %.5f' % sqrt(mean_squared_error(np.exp(full_model_ridge.predict(full_x_test)), np.exp(full_y_test))))
print('Gradient Regressor RMSE Score: %.5f' % sqrt(mean_squared_error(np.exp(full_model_gradient.predict(full_x_test)), np.exp(full_y_test))))


Simple Linear RMSE Score: 49.61277
Ridge RMSE Score: 50.37211
Gradient Regressor RMSE Score: 49.64466


### Top Features

In [2009]:
full_top = [i[0] for i in sorted(list(zip(full_x_train,full_model_gradient.feature_importances_)), key = lambda x: x[1], reverse = True)][:10]

In [2010]:
full_top

['zri_1y',
 'zri_2y',
 'median_rent_5y',
 'median_rent_4y',
 'zri_3y',
 'owner_occupied_housing_units_upper_value_quartile_5y',
 'income_per_capita_3y',
 'owner_occupied_housing_units_median_value_5y',
 'A03210_4y',
 'N18450_5y']

## Model with just top 10 most important features

In [2011]:
top_full_x_train = full_x_train[full_top]
top_full_x_test = full_x_test[full_top]

In [2012]:
top_full_y_train = full_y_train.copy()
top_full_y_test = full_y_test.copy()

In [2013]:
top_full_model_ols = LinearRegression()
top_full_model_ridge = Ridge()
top_full_model_gradient = GradientBoostingRegressor()

top_full_model_ols.fit(top_full_x_train, top_full_y_train)
top_full_model_ridge.fit(top_full_x_train, top_full_y_train)
top_full_model_gradient.fit(top_full_x_train, top_full_y_train)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
                          learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='auto',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

### R^2

In [2021]:
print('Simple Linear Test Score: %.4f'%top_full_model_ols.score(top_full_x_test, top_full_y_test))
print('Ridge Test Score: %.4f'%top_full_model_ridge.score(top_full_x_test, top_full_y_test))
print('Gradient Regressor Test Score: %.4f'%top_full_model_gradient.score(top_full_x_test, top_full_y_test))

Simple Linear Test Score: 0.9924
Ridge Test Score: 0.9923
Gradient Regressor Test Score: 0.9920


### RMSE

In [2041]:
print('Simple Linear RMSE Score: %.5f' % sqrt(mean_squared_error(np.exp(top_full_model_ols.predict(top_full_x_test)), np.exp(top_full_y_test))))
print('Ridge RMSE Score: %.5f' % sqrt(mean_squared_error(np.exp(top_full_model_ridge.predict(top_full_x_test)), np.exp(top_full_y_test))))
print('Gradient Regressor RMSE Score: %.5f' % sqrt(mean_squared_error(np.exp(top_full_model_gradient.predict(top_full_x_test)), np.exp(top_full_y_test))))


Simple Linear RMSE Score: 48.19237
Ridge RMSE Score: 48.78454
Gradient Regressor RMSE Score: 49.60003


## Voting Regressor (Census)

In [1374]:
from sklearn.ensemble import VotingRegressor

VR = VotingRegressor([('ols',model_ols),('rg', model_ridge),('gr',model_gradient)])

VR.fit(x_train, y_train)

VotingRegressor(estimators=[('ols',
                             LinearRegression(copy_X=True, fit_intercept=True,
                                              n_jobs=None, normalize=False)),
                            ('rg',
                             Ridge(alpha=1.0, copy_X=True, fit_intercept=True,
                                   max_iter=None, normalize=False,
                                   random_state=None, solver='auto',
                                   tol=0.001)),
                            ('gr',
                             GradientBoostingRegressor(alpha=0.9,
                                                       criterion='friedman_mse',
                                                       init=None,
                                                       learning_rate=0.1,
                                                       loss='ls', max_depth=3,
                                                       max_features=None,
                                  

### Train Score

In [1464]:
print('Voting Regressor Train Score: %.4f'%VR.score(x_train, y_train))

Voting Regressor Train Score: 0.9941


### Test Score

In [1465]:
print('Voting Regressor Test Score: %.4f'%VR.score(x_test, y_test))

Voting Regressor Test Score: 0.9924


### Kaggle Score

In [1466]:
print('Voting Regressor Kaggle Score: %.5f'  %sqrt(mean_squared_error(VR.predict(x_test), y_test)))

Voting Regressor Kaggle Score: 0.03147


### RMSE

In [1893]:
print('Voting Regressor Kaggle Score: %.5f'  %sqrt(mean_squared_error(np.exp(VR.predict(x_test)), np.exp(y_test))))

Voting Regressor Kaggle Score: 49.02019


### Accuracy

In [1452]:
VR_prd = pd.Series(np.exp(VR.predict(x_test)), index = y_test.index)

In [1471]:
print("Average Accuracy for VR Regressor is: %.2f" %((1 - np.abs(np.exp(y_test)/VR_prd - 1).mean())*100))

Average Accuracy for VR Regressor is: 97.65


## Voting Regressor (IRS)

In [1472]:
from sklearn.ensemble import VotingRegressor

irs_VR = VotingRegressor([('ols',irs_model_ols),('rg', irs_model_ridge),('gr',irs_model_gradient)])

irs_VR.fit(irs_x_train, irs_y_train)

VotingRegressor(estimators=[('ols',
                             LinearRegression(copy_X=True, fit_intercept=True,
                                              n_jobs=None, normalize=False)),
                            ('rg',
                             Ridge(alpha=1.0, copy_X=True, fit_intercept=True,
                                   max_iter=None, normalize=False,
                                   random_state=None, solver='auto',
                                   tol=0.001)),
                            ('gr',
                             GradientBoostingRegressor(alpha=0.9,
                                                       criterion='friedman_mse',
                                                       init=None,
                                                       learning_rate=0.1,
                                                       loss='ls', max_depth=3,
                                                       max_features=None,
                                  

In [1477]:
print('Voting Regressor Train Score: %.4f'%irs_VR.score(irs_x_train, irs_y_train))

Voting Regressor Train Score: 0.9951


In [1478]:
print('Voting Regressor Test Score: %.4f'%irs_VR.score(irs_x_test, irs_y_test))

Voting Regressor Test Score: 0.9921


In [1479]:
print('Voting Regressor Kaggle Score: %.5f'  %sqrt(mean_squared_error(irs_VR.predict(irs_x_test), irs_y_test)))

Voting Regressor Kaggle Score: 0.03206


In [2037]:
print('Average RMSE: %.5f'  %sqrt(mean_squared_error(np.exp(irs_VR.predict(irs_x_test)), np.exp(irs_y_test))))

Average RMSE: 50.68215


In [1558]:
irs_VR_prd = pd.Series(np.exp(irs_VR.predict(irs_x_test)), index = irs_y_test.index)


print("Average Accuracy for VR Regressor is: %.2f" %((1 - np.abs(np.exp(irs_y_test)/irs_VR_prd - 1).mean())*100))


Average Accuracy for VR Regressor is: 97.61


## Average of 2 Voting Regressors

In [1898]:
VR_together = pd.concat([VR_prd, irs_VR_prd], join = 'inner',axis = 1)

In [1899]:
VR_together.columns = ['census', 'irs']

In [1900]:
VR_together['average'] = (VR_together.census + VR_together.irs)/2

In [1904]:
VR_together = pd.concat([VR_together.average,np.exp(y_test)], join = 'inner', axis = 1)

In [2240]:
print("Average RMSE: %.3f" %sqrt(mean_squared_error(VR_together.average, VR_together['ZRI+3'])))

Average RMSE: 48.424


In [2241]:
r2_score(VR_together['ZRI+3'],VR_together.average)

0.9935133773565724

In [1910]:
(1 - np.abs(np.exp(y_test)/VR_together.average - 1)).mean()*100

97.69578632070164

## Voting Regressor for Combined Census and IRS

In [2038]:
from sklearn.ensemble import VotingRegressor

full_VR = VotingRegressor([('ols',full_model_ols),('rg', full_model_ridge),('gr', full_model_gradient)])

full_VR.fit(full_x_train, full_y_train)

VotingRegressor(estimators=[('ols',
                             LinearRegression(copy_X=True, fit_intercept=True,
                                              n_jobs=None, normalize=False)),
                            ('rg',
                             Ridge(alpha=1.0, copy_X=True, fit_intercept=True,
                                   max_iter=None, normalize=False,
                                   random_state=None, solver='auto',
                                   tol=0.001)),
                            ('gr',
                             GradientBoostingRegressor(alpha=0.9,
                                                       criterion='friedman_mse',
                                                       init=None,
                                                       learning_rate=0.1,
                                                       loss='ls', max_depth=3,
                                                       max_features=None,
                                  

### R^2

In [2039]:
print('Voting Regressor Test Score: %.4f'%full_VR.score(full_x_test, full_y_test))

Voting Regressor Test Score: 0.9923


### RMSE

In [2036]:
print('Average RMSE: %.5f'  %sqrt(mean_squared_error(np.exp(full_VR.predict(full_x_test)), np.exp(full_y_test))))


Average RMSE: 49.23186


## Voting Regressor for Top 10 Combined

In [2032]:
from sklearn.ensemble import VotingRegressor

top_full_VR = VotingRegressor([('ols',top_full_model_ols),('rg', top_full_model_ridge),('gr', top_full_model_gradient)])

top_full_VR.fit(top_full_x_train, top_full_y_train)


VotingRegressor(estimators=[('ols',
                             LinearRegression(copy_X=True, fit_intercept=True,
                                              n_jobs=None, normalize=False)),
                            ('rg',
                             Ridge(alpha=1.0, copy_X=True, fit_intercept=True,
                                   max_iter=None, normalize=False,
                                   random_state=None, solver='auto',
                                   tol=0.001)),
                            ('gr',
                             GradientBoostingRegressor(alpha=0.9,
                                                       criterion='friedman_mse',
                                                       init=None,
                                                       learning_rate=0.1,
                                                       loss='ls', max_depth=3,
                                                       max_features=None,
                                  

### R^2

In [2033]:
print('Voting Regressor Test Score: %.4f'%top_full_VR.score(top_full_x_test, top_full_y_test))

Voting Regressor Test Score: 0.9924


### RMSE

In [2043]:
print('Average RMSE: %.5f'  %sqrt(mean_squared_error(np.exp(top_full_VR.predict(top_full_x_test)), np.exp(top_full_y_test))))


Average RMSE: 48.34092


## Voting Regressor Top 10 Census

In [2073]:
from sklearn.ensemble import VotingRegressor

top_census_VR = VotingRegressor([('ols',top_model_ols),('rg', top_model_ridge),('gr', top_model_gradient)])

top_census_VR.fit(top_x_train, top_y_train)


VotingRegressor(estimators=[('ols',
                             LinearRegression(copy_X=True, fit_intercept=True,
                                              n_jobs=None, normalize=False)),
                            ('rg',
                             Ridge(alpha=1.0, copy_X=True, fit_intercept=True,
                                   max_iter=None, normalize=False,
                                   random_state=None, solver='auto',
                                   tol=0.001)),
                            ('gr',
                             GradientBoostingRegressor(alpha=0.9,
                                                       criterion='friedman_mse',
                                                       init=None,
                                                       learning_rate=0.1,
                                                       loss='ls', max_depth=3,
                                                       max_features=None,
                                  

### R^2

In [2075]:
print('Voting Regressor Test Score: %.4f'%top_census_VR.score(top_x_test, top_y_test))

Voting Regressor Test Score: 0.9921


### RMSE

In [2076]:
print('Average RMSE: %.5f'  %sqrt(mean_squared_error(np.exp(top_census_VR.predict(top_x_test)), np.exp(top_y_test))))


Average RMSE: 48.01541


## Voting Regressor Top 10 IRS

In [2078]:
from sklearn.ensemble import VotingRegressor

top_irs_VR = VotingRegressor([('ols',top_irs_model_ols),('rg', top_irs_model_ridge),('gr', top_irs_model_gradient)])

top_irs_VR.fit(top_irs_x_train, top_irs_y_train)


VotingRegressor(estimators=[('ols',
                             LinearRegression(copy_X=True, fit_intercept=True,
                                              n_jobs=None, normalize=False)),
                            ('rg',
                             Ridge(alpha=1.0, copy_X=True, fit_intercept=True,
                                   max_iter=None, normalize=False,
                                   random_state=None, solver='auto',
                                   tol=0.001)),
                            ('gr',
                             GradientBoostingRegressor(alpha=0.9,
                                                       criterion='friedman_mse',
                                                       init=None,
                                                       learning_rate=0.1,
                                                       loss='ls', max_depth=3,
                                                       max_features=None,
                                  

### R^2

In [2080]:
print('Voting Regressor Test Score: %.4f'%top_irs_VR.score(top_irs_x_test, top_irs_y_test))

Voting Regressor Test Score: 0.9921


### RMSE

In [2081]:
print('Average RMSE: %.5f'  %sqrt(mean_squared_error(np.exp(top_irs_VR.predict(top_irs_x_test)), np.exp(top_irs_y_test))))


Average RMSE: 50.18733


## Average of top 10 Voting Regressors

In [2090]:
top_VR_census_pred = pd.Series(np.exp(top_census_VR.predict(top_x_test)), index = top_x_test.index)

In [2091]:
top_VR_irs_pred = pd.Series(np.exp(top_irs_VR.predict(top_irs_x_test)), index = top_irs_x_test.index)

In [2098]:
total_top_VR = pd.concat([top_VR_census_pred,top_VR_irs_pred], join = 'inner', axis = 1)
total_top_VR.columns = ['census_vr','irs_vr']
total_top_VR['average_vr'] = (total_top_VR.census_vr + total_top_VR.irs_vr)/2
total_top_VR = pd.concat([total_top_VR, np.exp(top_y_test)], join = 'inner', axis = 1)

### RMSE

In [2242]:
print('Average RMSE: %.5f'  %sqrt(mean_squared_error(total_top_VR['average_vr'], total_top_VR['ZRI+3'])))


Average RMSE: 47.19771


In [2243]:
r2_score(total_top_VR['ZRI+3'],total_top_VR['average_vr'])

0.9938377760388308

# Second Level Model (based on census and irs predictions)

In [1457]:
together = pd.concat([together,np.exp(y_test)], join = 'inner', axis = 1)

In [1458]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(together.iloc[:,:2], together.iloc[:,3], test_size=0.20, random_state=42)

In [1459]:
X_train = np.log(X_train)
X_test = np.log(X_test)
Y_train = np.log(Y_train)
Y_test = np.log(Y_test)

In [1460]:
second_model_ols = LinearRegression()
second_model_ridge = Ridge()
second_model_gradient = GradientBoostingRegressor()

second_model_ols.fit(X_train, Y_train)
second_model_ridge.fit(X_train, Y_train)
second_model_gradient.fit(X_train, Y_train)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
                          learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='auto',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [1461]:
print(second_model_ols.score(X_train, Y_train))
print(second_model_ridge.score(X_train, Y_train))
print(second_model_gradient.score(X_train, Y_train))

0.9928272033748813
0.9928261802322604
0.9935607575883438


In [1462]:
print(second_model_ols.score(X_test, Y_test))
print(second_model_ridge.score(X_test, Y_test))
print(second_model_gradient.score(X_test, Y_test))

0.9923927436413895
0.9923827091879923
0.9922117275494151


In [1463]:
print(sqrt(mean_squared_error(second_model_ols.predict(X_test), Y_test)))
print(sqrt(mean_squared_error(second_model_ridge.predict(X_test), Y_test)))
print(sqrt(mean_squared_error(second_model_gradient.predict(X_test), Y_test)))

0.03123234217610667
0.0312529341074772
0.03160174729959855


In [1484]:
(1 - np.abs(np.exp(Y_test)/pd.Series(np.exp(second_model_ols.predict(X_test)), index = Y_test.index) - 1).mean())*100



97.71715692029895

# Second Level Model (based on Voting Regressors of Census and IRS)

In [1594]:
VR_together = pd.concat([VR_together,np.exp(y_test)], join = 'inner', axis = 1)

In [1595]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(VR_together.iloc[:,:2], VR_together.iloc[:,3], test_size=0.20, random_state=42)

X_train = np.log(X_train)
X_test = np.log(X_test)
Y_train = np.log(Y_train)
Y_test = np.log(Y_test)

second_model_ols = LinearRegression()
second_model_ridge = Ridge()
second_model_gradient = GradientBoostingRegressor()

second_model_ols.fit(X_train, Y_train)
second_model_ridge.fit(X_train, Y_train)
second_model_gradient.fit(X_train, Y_train)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
                          learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='auto',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [1596]:
print(second_model_ols.score(X_train, Y_train))
print(second_model_ridge.score(X_train, Y_train))
print(second_model_gradient.score(X_train, Y_train))

0.9928287381021246
0.992826410275961
0.993618028628102


In [1597]:
print(second_model_ols.score(X_test, Y_test))
print(second_model_ridge.score(X_test, Y_test))
print(second_model_gradient.score(X_test, Y_test))

0.9924471754108709
0.9924333781323569
0.9920866483413247


In [1598]:
print(sqrt(mean_squared_error(second_model_ols.predict(X_test), Y_test)))
print(sqrt(mean_squared_error(second_model_ridge.predict(X_test), Y_test)))
print(sqrt(mean_squared_error(second_model_gradient.predict(X_test), Y_test)))

0.031120404078258723
0.031148816029516177
0.031854497675436674


In [1600]:
(1 - np.abs(np.exp(Y_test)/pd.Series(np.exp(second_model_ols.predict(X_test)), index = Y_test.index) - 1).mean())*100

97.73532606189075